# 0.Libraray Import & Data Load
- 유동인구 데이터 (SKT)
- 카드데이터 (신한카드)
- 유통데이터 (편의점, GS)

- 외부데이터
  - 코로나데이터 (코로나 확진자 수 데이터)
  - 날씨데이터 (서울 대구, 평균기온, 일강수량, 평균상대습도)
  - 휴일 여부 데이터 (2020, 2019)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터 분석 library
import pandas as pd
import numpy as np

# 통계용 library
import scipy
from scipy.stats.stats import pearsonr # 상관계수
import statsmodels.formula.api as smf # 회귀모델용

!pip install pingouin
import pingouin as pg # 편상관분석용

# 기타
from functools import reduce
import datetime
from datetime import datetime, timedelta
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 225kB 3.4MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.8-cp36-none-any.whl size=221688 sha256=226a06989fca462addd15ae5a3a3783666b7aee82be69e2ae5adbbc00793e1b0
  Stored in directory: /root/.cache/pip/wheels/d6/9e/53/f885f73f29cf7c8cac3d8f4b1532bbfef2f5eb543946ac9055
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=4e85e93243d7fb90f42e41ac8aff4ea81506e2f529ff89cd9eeb00cfc2a63b8c
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=c38a15260ffc0f40a727ffe558a45e9cfb103757892e32887dbbb4505d5003b5
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


## 유동인구 데이터 (2020, 2019)
- SKT 유동인구 데이터 2020, 2019 사용

In [3]:
# 2020년 데이터
skt_age_202002 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202002.CSV", sep="|")
skt_age_202003 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202003.CSV", sep="|")
skt_age_202004 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202004.CSV", sep="|")
skt_age_202005 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202005.CSV", sep="|")

# 2019 유동인구 데이터 age
skt_age_201902=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201902.CSV", sep="|")
skt_age_201903=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201903.CSV", sep="|")
skt_age_201904=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201904.CSV", sep="|")
skt_age_201905=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201905.CSV", sep="|")

# skt 데이터 합침 (2020, 2019)
skt_2020 = pd.concat([skt_age_202002,skt_age_202003,skt_age_202004,skt_age_202005]) # 2020년 데이터
skt_2019 = pd.concat([skt_age_201902, skt_age_201903, skt_age_201904, skt_age_201905]) # 2019년 데이터

print(skt_2020.shape, skt_2019.shape)
skt_2020

(8349, 34) (8280, 34)


,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,MAN_FLOW_POP_CNT_4044,MAN_FLOW_POP_CNT_4549,MAN_FLOW_POP_CNT_5054,MAN_FLOW_POP_CNT_5559,MAN_FLOW_POP_CNT_6064,MAN_FLOW_POP_CNT_6569,MAN_FLOW_POP_CNT_70U,WMAN_FLOW_POP_CNT_0004,WMAN_FLOW_POP_CNT_0509,WMAN_FLOW_POP_CNT_1014,WMAN_FLOW_POP_CNT_1519,WMAN_FLOW_POP_CNT_2024,WMAN_FLOW_POP_CNT_2529,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U
0,202002,20200201,1114052000,소공동,0.0,88.95,376.68,1077.03,3602.41,6660.39,6519.52,5675.54,5063.35,5992.92,6207.66,5997.45,6356.51,4740.35,5766.20,0.0,73.78,433.73,1408.06,4999.85,6961.02,5115.24,4164.47,3927.93,4058.47,3932.39,3650.70,3361.05,2440.61,2665.43
1,202002,20200202,1114052000,소공동,0.0,51.03,271.96,871.27,2824.97,5031.63,4626.14,3975.63,3455.81,4218.84,4473.19,4183.04,4278.93,3182.24,3567.28,0.0,55.15,327.72,1231.75,3807.14,4702.91,3698.04,2922.33,2781.63,2947.01,2632.34,2669.56,2399.90,1704.22,2026.77
2,202002,20200203,1114052000,소공동,0.0,32.47,136.94,826.26,3580.23,8823.11,11803.45,14934.31,14823.99,16430.05,14201.85,10908.99,9628.67,6658.04,7411.35,0.0,35.09,155.99,1166.65,6301.35,11109.59,10754.75,9655.01,9018.58,8786.59,7257.35,5710.55,4935.00,3070.67,3349.99
3,202002,20200204,1114052000,소공동,0.0,27.34,154.75,823.69,3510.75,9361.85,12127.55,15650.18,15484.99,16854.01,14653.71,11550.17,9567.69,6884.62,7319.09,0.0,40.41,193.20,1148.80,6345.14,11771.06,11467.86,9834.22,9323.74,8982.67,7448.09,5938.57,4948.10,3091.30,3494.91
4,202002,20200205,1114052000,소공동,0.0,26.30,177.28,809.21,3579.35,9631.85,12378.43,15876.88,15624.94,16730.33,14462.22,11456.97,9360.22,6709.08,6951.99,0.0,36.44,172.20,1285.73,6587.78,11876.40,11285.83,9972.17,9299.52,8976.55,7183.79,5719.89,4822.39,2960.73,3204.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,202005,20200527,2726069000,고산3동,0.0,70.07,519.05,956.45,1205.44,1507.57,1500.51,2014.87,2295.21,2947.78,3020.17,2653.97,2217.68,1561.13,1492.80,0.0,114.34,552.87,842.82,1264.78,1201.14,1106.57,1728.34,2209.31,2808.54,2721.54,2070.62,1613.50,1086.34,1143.37
2135,202005,20200528,2726069000,고산3동,0.0,99.04,749.36,1085.68,1639.82,1777.32,1776.02,2137.50,2582.56,3415.30,3458.23,3019.66,2550.45,1845.30,1780.62,0.0,146.62,733.55,1117.85,1597.98,1377.94,1258.02,1847.61,2677.19,3342.68,3036.71,2389.70,1848.92,1291.19,1515.36
2136,202005,20200529,2726069000,고산3동,0.0,155.70,781.70,1235.94,1706.97,1982.74,1924.60,2500.02,2810.11,3625.34,3718.35,3212.18,2852.53,1755.48,1811.35,0.0,111.30,801.94,1167.51,1709.27,1583.58,1487.06,2149.30,2916.87,3643.31,3395.24,2800.02,2043.25,1356.15,1455.89
2137,202005,20200530,2726069000,고산3동,0.0,119.89,903.85,1236.60,1722.01,2142.41,2106.49,2215.86,2649.30,3713.62,3854.18,3138.66,2506.90,1683.99,1666.55,0.0,152.58,727.46,1229.17,1627.58,1644.73,1420.31,1812.27,2640.71,3362.25,2909.34,2353.82,1812.56,1242.56,1372.38


In [4]:
# skt 유동인구 데이터 - 전처리 (2019를 2020과 날짜를 맞추어 줌)

# 2020 유동인구 데이터 연령 총합 변수 만들기
skt_2020['sum_20'] = skt_2020.iloc[:,4:].sum(axis=1)
skt_2020 = skt_2020[['STD_YMD','HDONG_NM','sum_20']]
skt_2020.columns = ['STD_DD','HDONG_NM','sum_20']
skt_2020 = skt_2020.reset_index(drop=True)

# 2019 유동인구 데이터 연령 총합 변수 만들기
skt_2019["sum_19"] = skt_2019.iloc[:,4:].sum(axis=1)
skt_2019 = skt_2019[["STD_YMD","HDONG_NM","sum_19"]]
skt_2019.columns = ['STD_DD','HDONG_NM','sum_19']
skt_2019 = skt_2019.reset_index(drop=True)

print(skt_2019.shape, skt_2019.shape)
skt_2019

(8280, 3) (8280, 3)


,STD_DD,HDONG_NM,sum_19
0,20190201,소공동,242230.68
1,20190202,소공동,133411.99
2,20190203,소공동,90124.93
3,20190204,소공동,83378.55
4,20190205,소공동,111009.47
...,...,...,...
8275,20190527,고산3동,47011.48
8276,20190528,고산3동,49344.74
8277,20190529,고산3동,50345.41
8278,20190530,고산3동,50148.22


## 카드데이터 (2020, 2019)
- 신한카드 데이터 (외국인 + 내국인)
- 신한카드 행정동 데이터
- 카드 데이터 + 행정동 데이터 Merge
- 2020, 2019 데이터 분리
- 카드데이터 업종별 대분류 리스트 변수 생성

In [5]:
# 신한 카드 데이터 (내국인, 외국인)
card_re = pd.read_csv('/content/drive/My Drive/big_contest/shinhan/CARD_SPENDING_RESIDENT.txt', sep = "\t", engine='python', encoding = "cp949")
card_fo = pd.read_csv('/content/drive/My Drive/big_contest/shinhan/CARD_SPENDING_FOREIGNER.txt', sep = "\t", engine='python', encoding = "cp949")

# 신한 카드 행정동 데이터
dong = pd.read_excel('/content/drive/My Drive/big_contest/shinhan/02_혁신아이디어분야_카드소비데이터(신한카드)_데이터정의서.xlsx', sheet_name='(참고) 행정동코드',index_col=0, header=None)

print(card_re.shape, card_fo.shape, dong.shape)

(2537753, 8) (238829, 7) (71, 4)


In [6]:
# 카드 행정동 데이터 전처리
dong.columns = ['GU_CD','DONG_CD','GU_NM','HDONG_NM']
dong.reset_index(inplace=True, drop=True)
dong = dong.loc[2:]
dong = dong.reset_index(drop=True)

# 컬럼 type int 형으로 변경
dong['GU_CD'] = dong['GU_CD'].astype(int)
dong['DONG_CD'] = dong['DONG_CD'].astype(int)

# 카드 데이터 (내국인 + 외국인) concat
card = pd.concat([card_fo,card_re])

# 행정동 이름과 카드 데이터 merge
card = pd.merge(card, dong, on=['GU_CD','DONG_CD'])

print(card.shape)
card

(2776582, 11)


,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,COUNTRY_NM,USE_CNT,USE_AMT,SEX_CD,AGE_CD,GU_NM,HDONG_NM
0,20190204,260,510,81,대만,2,30.0,NaN,NaN,대구 수성구,범어1동
1,20190213,260,510,40,대만,2,32.0,NaN,NaN,대구 수성구,범어1동
2,20190416,260,510,40,대만,2,16.0,NaN,NaN,대구 수성구,범어1동
3,20190419,260,510,42,대만,3,610.0,NaN,NaN,대구 수성구,범어1동
4,20190308,260,510,10,대만,7,629.0,NaN,NaN,대구 수성구,범어1동
...,...,...,...,...,...,...,...,...,...,...,...
2776577,20200531,350,710,81,NaN,47,570.0,M,45.0,서울 노원구,상계9동
2776578,20200531,350,710,81,NaN,33,444.0,M,50.0,서울 노원구,상계9동
2776579,20200531,350,710,81,NaN,28,403.0,M,55.0,서울 노원구,상계9동
2776580,20200531,350,710,81,NaN,33,384.0,M,60.0,서울 노원구,상계9동


In [7]:
# 일별, 구별 ,동별 USE_AMT, USE_AMT 합
card_group = card.groupby(["STD_DD",'GU_NM','HDONG_NM'], as_index=False)['USE_CNT','USE_AMT'].agg("sum")

# 2020, 2019 카드 데이터 분리
card_2020 = card_group[card_group.STD_DD>=20200101] # 2020년 카드 데이터
card_2019 = card_group[card_group.STD_DD<20200101] # 2019년 카드 데이터

print(card_2020.shape, card_2019.shape)

(8349, 5) (8280, 5)


In [8]:
# 카드 대분류 list dictionary
type_list = {'10':'숙박(10)', '20':'레저용품(20)', '21':'레저업소(21)', 
            '22':'문화취미(22)', '30':'가구(30)' , '31':'전기(31)' ,
            '32':'주방용구(32)', '33':'연료판매(33)' , '34':'광학제품(34)',
            '35':'가전(35)', '40':'유통업(40)', '42':'의복(42)', 
            '43':'직물(43)', '44':'신변잡화(44)', '50':'서적문구(50)',
            '52':'사무통신(52)', '60':'자동차판매(60)','62':'자동차정비(62)',
            '70':'의료기관(70)', '71':'보건위생(71)', '80':'요식업소(80)',
            '81':'음료식품(81)', '92':'수리서비스(92)'}

# 업종 list
MCT_CAT_CD_list = card['MCT_CAT_CD'].unique()
MCT_CAT_CD_list = sorted(MCT_CAT_CD_list)
MCT_CAT_CD_list = list(map(str, MCT_CAT_CD_list))
print(len(MCT_CAT_CD_list), MCT_CAT_CD_list)

23 ['10', '20', '21', '22', '30', '31', '32', '33', '34', '35', '40', '42', '43', '44', '50', '52', '60', '62', '70', '71', '80', '81', '92']


## 편의점(유통, GS) 데이터 (2020,2019)
- GS 제공 편의점 데이터
- 구조방정식, 매개효과용 데이터 변수 : gs
- 회귀용 데이터 변수명 : gs_row -> 2020, 2019 분리해서 사용

In [9]:
# GS 데이터 가져오기
gs = pd.read_excel('/content/drive/My Drive/big_contest/GS/유통(GS리테일).xlsx', sheet_name='종합테이블(TB_DD_STR_PURCH_CLS)')
print(gs.shape)
gs

(14460, 13)


,OPER_DT,PVN,BOR,ADMD,AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P
0,20190201,대구광역시,수성구,고산1동,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,20190201,대구광역시,수성구,고산2동,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,20190201,대구광역시,수성구,고산3동,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,20190201,대구광역시,수성구,두산동,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,20190201,대구광역시,수성구,만촌1동,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14455,20200531,서울특별시,중구,중림동,1.595038,0.314217,0.221087,0.344307,0.026204,0.070094,0.006347,0.013541,0.004203
14456,20200531,서울특별시,중구,청구동,1.389184,0.276333,0.221216,0.363681,0.053907,0.076963,0.001592,0.000353,0.005955
14457,20200531,서울특별시,중구,필동,0.714265,0.303338,0.187615,0.370371,0.038908,0.079408,0.007826,0.008298,0.004236
14458,20200531,서울특별시,중구,황학동,1.134190,0.264708,0.257387,0.367001,0.029140,0.070770,0.001060,0.003646,0.006287


In [10]:
# 차후 군집별 업종 상관관계, 회귀 모델용 데이터 변수
gs_row = gs.copy()

# GS 데이터 필요 column 줄이기 (구조방정식, 매개효과 용 변수)
gs.columns=['STD_DD', 'HGU_NM', 'CTGG_NM', 'HDONG_NM', 'AMT_IND', 'LCLS_10_P', 'LCLS_20_P',
            'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P', 'LCLS_80_P']
gs = gs[['STD_DD', 'HGU_NM', 'HDONG_NM', 'AMT_IND']]
gs

,STD_DD,HGU_NM,HDONG_NM,AMT_IND
0,20190201,대구광역시,고산1동,1.064336
1,20190201,대구광역시,고산2동,1.076418
2,20190201,대구광역시,고산3동,0.856071
3,20190201,대구광역시,두산동,0.724868
4,20190201,대구광역시,만촌1동,1.155728
...,...,...,...,...
14455,20200531,서울특별시,중림동,1.595038
14456,20200531,서울특별시,청구동,1.389184
14457,20200531,서울특별시,필동,0.714265
14458,20200531,서울특별시,황학동,1.134190


In [11]:
# 유통 GS 품목별 대분류 list dictionary
circulation_list = {'mul_LCLS_10_P':'식사',
                    'mul_LCLS_20_P':'간식',
                    'mul_LCLS_30_P':'마실거리',
                    'mul_LCLS_40_P':'홈&리빙',
                    'mul_LCLS_50_P':'헬스&뷰티',
                    'mul_LCLS_60_P':'취미&여가활동_비중',
                    'mul_LCLS_70_P':'사회활동_비중',
                    'mul_LCLS_80_P':'임신/육아_비중',
                    'mul_LCLS_90_P':'기호품_비중'}

# 유통 GS 품목별 대분류 list dictionary
circulation_list_reverse = {'식사':               'mul_LCLS_10_P',
                            '간식':               'mul_LCLS_20_P',
                            '마실거리' :          'mul_LCLS_30_P',
                            '홈&리빙':            'mul_LCLS_40_P',
                            '헬스&뷰티':          'mul_LCLS_50_P',
                            '취미&여가활동_비중': 'mul_LCLS_60_P',
                            '사회활동_비중':      'mul_LCLS_70_P',
                            '임신/육아_비중':     'mul_LCLS_80_P',
                            '기호품_비중':        'mul_LCLS_90_P'}

## 코로나 데이터 (2020)
- 출처 : https://github.com/jooeungen/coronaboard_kr // CoronaBoard_kr 깃허브 
- 출처 : http://ncov.mohw.go.kr/ // corona board에서 참조한 질병관리본부 링크
- <b>CoronaBoard data - Available Commercial & Non-commercial</b>
  - kr_daily.csv (코로나 일일 확진자 수 - 누적) (원본)
  - kr_regional_data.csv (코로나 지역별  일일 확진자 수 - 누적 )(원본)

- <b>사용데이터 : 코로나 일일 확진자 수 데이터 (전국, 서울, 대구)</b>
  - <b>위 출처 데이터를 1차 가공(차분)한 데이터 (누적 -> 일일) </b>
  - 기간 : 20200201 ~ 20200601 데이터 사용
  - covid_all.csv : 전국 코로나 일일 확진자 수
  - corona_seoul.csv : 서울지역 코로나 일일 확진자 수
  - corona_daegu.csv : 대구지역 코로나 일일 확진자 수
  - 코로나 데이터는 하루씩 당겨서 사용
  

In [12]:
# 코로나 전국 확진자 데이터
covid = pd.read_csv("/content/drive/My Drive/big_contest/corona/covid_all.csv", index_col=0, encoding="cp949")

# 서울, 대구 각각의 지역 확진자 데이터
covid_seoul = pd.read_csv("/content/drive/My Drive/big_contest/corona/corona_seoul.csv", index_col=0, encoding="cp949")
covid_daegu = pd.read_csv("/content/drive/My Drive/big_contest/corona/corona_daegu.csv", index_col=0, encoding="cp949")

# 전국 코로나 데이터
covid = covid.reset_index()
covid.columns = ['STD_DD','confirmed']
covid

,STD_DD,confirmed
0,20200201,4
1,20200202,1
2,20200203,3
3,20200204,0
4,20200205,1
...,...,...
179,20200729,28
180,20200730,48
181,20200731,18
182,20200801,36


In [13]:
# corona 지역 데이터 (서울, 대구)
covid_seoul = covid_seoul[['date','confirmed_diff']]
covid_seoul.columns=['STD_DD','seoul_confirmed']
covid_daegu = covid_daegu[['date','confirmed_diff']]
covid_daegu.columns=['STD_DD','daegu_confirmed']

# 서울, 대구 코로나 데이터
print(covid_seoul.shape, covid_daegu.shape)
covid_seoul.head(3)

(121, 2) (121, 2)


,STD_DD,seoul_confirmed
1,20200201,0
2,20200202,1
3,20200203,0


## 날씨데이터 (2020)
- 기상청에서 서울지역과 대구지역 2020 날씨 Download
- 출처 : https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36
- 서울 지역과 대구지역 기상 데이터
- 평균기온, 일강수량, 평균상대습도만 사용

In [14]:
# 서울지역과 대구지역 날씨 
seoul_weather = pd.read_csv('/content/drive/My Drive/big_contest/weather/2020서울날씨.csv')
daegu_weather = pd.read_csv('/content/drive/My Drive/big_contest/weather/2020_대구_날씨.csv')
print(seoul_weather.shape, daegu_weather.shape)
seoul_weather

(121, 10) (121, 10)


,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 상대습도(%),합계 일사량(MJ/m2),평균 전운량(1/10)
0,108,서울,2020-02-01,2.6,-1.2,7.6,NaN,66.6,7.17,3.4
1,108,서울,2020-02-02,1.5,-2.1,5.3,NaN,52.3,8.39,6.1
2,108,서울,2020-02-03,-0.4,-2.9,3.0,0.0,43.6,13.56,1.5
3,108,서울,2020-02-04,-1.6,-6.0,3.9,1.7,50.3,10.01,3.9
4,108,서울,2020-02-05,-8.3,-11.0,-4.9,NaN,37.3,14.82,0.0
...,...,...,...,...,...,...,...,...,...,...
116,108,서울,2020-05-27,17.4,11.8,23.9,NaN,68.5,30.06,1.5
117,108,서울,2020-05-28,18.7,14.2,23.4,0.0,62.8,22.83,3.9
118,108,서울,2020-05-29,20.7,14.6,27.6,NaN,65.6,28.09,1.3
119,108,서울,2020-05-30,22.6,16.5,30.0,NaN,57.3,26.50,1.1


In [15]:
# 사용할 컬럼 선택 및 컬럼명 변경
seoul_weather = seoul_weather[['일시','평균기온(°C)', '일강수량(mm)', '평균 상대습도(%)']]
seoul_weather.columns = ['STD_DD','temp_mean','rain_mean','hum_mean']

daegu_weather = daegu_weather[['일시','평균기온(°C)', '일강수량(mm)', '평균 상대습도(%)']]
daegu_weather.columns = ['STD_DD','temp_mean','rain_mean','hum_mean']

# 날씨 표현에서 - 없애주기
seoul_weather['STD_DD'] = seoul_weather['STD_DD'].str.replace('-','')
daegu_weather['STD_DD'] = daegu_weather['STD_DD'].str.replace('-','')
seoul_weather['STD_DD'] = seoul_weather['STD_DD'].astype(int)
daegu_weather['STD_DD'] = daegu_weather['STD_DD'].astype(int)

# 강수량만 null 값 존재. (측정안된 것 - 비가 안온 것 -> 0으로 처리)
seoul_weather['rain_mean'] = seoul_weather['rain_mean'].fillna(0)
daegu_weather['rain_mean'] = seoul_weather['rain_mean'].fillna(0)

print(seoul_weather.shape, daegu_weather.shape)

(121, 4) (121, 4)


## 휴일여부 데이터 (2019, 2020)
- 2019년, 2020년 휴일 여부 데이터
- 직접 데이터 수작업 작성
- 명절, 휴일, 대체휴일 등 반영

In [16]:
# 2019, 2020 휴일 여부 데이터
holiday = pd.read_csv("/content/drive/My Drive/big_contest/holiday/holiday.csv")

print(holiday.shape)
holiday

(241, 3)


,date,day,weekend
0,20190201,4,0
1,20190202,5,1
2,20190203,6,1
3,20190204,0,1
4,20190205,1,1
...,...,...,...
236,20200527,2,0
237,20200528,3,0
238,20200529,4,0
239,20200530,5,1


In [17]:
# 컬럼명 변경, type 변경
holiday.columns = ['STD_DD','day','weekend']

holiday['STD_DD'] = holiday['STD_DD'].astype(str)
holiday['STD_DD'] = holiday['STD_DD'].str.replace("-","")
holiday['STD_DD'] = holiday['STD_DD'].astype(int)
holiday

,STD_DD,day,weekend
0,20190201,4,0
1,20190202,5,1
2,20190203,6,1
3,20190204,0,1
4,20190205,1,1
...,...,...,...
236,20200527,2,0
237,20200528,3,0
238,20200529,4,0
239,20200530,5,1


## 서울 대구 행정동 리스트
- 서울 중구/노원구 행정동 리스트
- 대구 중구/수성구 행정동 리스트

In [18]:
# 분석의 편의를 위해 서울시 중구, 서울시 노원구, 대구시 중구, 대구시 수성구 동별로 리스트 생성
seoul_jung_list = ['소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동', '신당동', '다산동','약수동', '청구동', '신당5동', '동화동', '황학동', '중림동']
seoul_nowon_list = ['월계1동', '월계2동', '월계3동','공릉1동', '공릉2동', '하계1동', '하계2동', '중계본동', '중계1동', '중계4동', '중계2.3동','상계1동', '상계2동', '상계3.4동', '상계5동', '상계6.7동', '상계8동', '상계9동','상계10동']
daegu_jung_list = ['동인동', '삼덕동', '성내1동', '성내2동', '성내3동', '대신동', '남산1동', '남산2동', '남산3동', '남산4동', '대봉1동', '대봉2동']
daegu_suseong_list = ['범어1동', '범어2동', '범어3동', '범어4동', '만촌1동', '만촌2동', '만촌3동', '수성1가동', '수성2.3가동', '수성4가동', '황금1동', 
                      '황금2동', '중동', '상동', '파동', '두산동', '지산1동', '지산2동', '범물1동', '범물2동', '고산1동', '고산2동', '고산3동']

# 5.최종 모델링 (POC 사용모델)
- 종속변수 업종/품목 별 매출액을 예측하기 위해, 일일 코로나 확진자 수, 기상자료 개방포털에서 제공하는 날씨 데이터 (기온, 강수량, 상대습도)를 독립변수로 사용하여 회귀 예측 진행
- 독립변수 명 :  C : 코로나(Corona) /  T: 기온(Temperature) / R: 강우량(Rainfall)  /  H : 습도(Humid)

- 모델의 사용되는 독립변수 조합에 따라 총 8가지의 모델이 발생 (C, CT, CH ... CTR, CTRH)

- 모든 독립변수들의 p-value 가 유의하고, 모델의 R squared 값이 가장 높은 경우의 최적 모델 유형 선택
- 업종/품목마다 다른 유형의 모델이 적용되며, 모델의 R squared 값이 0.35 이상인 업종/품목 최종 선택

- 결과 csv 파일들 (32개)
  - 카드서울평일군집1~4 : 4개
  - 카드서울휴일군집1~4 : 4개
  - 카드대구평일군집1~4 : 4개
  - 카드대구휴일군집1~4 : 4개
  - 편의점서울평일군집1~4 : 4개
  - 편의점서울휴일군집1~4 : 4개
  - 편의점대구평일군집1~4 : 4개
  - 편의점대구휴일군집1~4 : 4개

- CSV 결과 해석
  - ([0.0, 0.0002, 0.4587], 0.203, [46.2506, -0.01, 0.03]
  - P-value / R squared / 회귀계수 값들 순

## 군집리스트 & 매개효과, 부트스트래핑 통과 리스트, 군집별 유의업종 리스트

In [19]:
# 서울, 대구 군집 리스트
# 1군집 (상권중심지역) /  2군집 (주거상권지역) / 3군집 (주거중심지역) / 4군집 (교통중심지역)

# 서울 cluster 1,2,3,4
seoul_cluster1 = ['명동', '상계2동', '소공동', '필동', '회현동']
seoul_cluster2 = ['동화동', '상계1동', '상계9동', '월계1동', '중계1동', '중계2.3동', '중계4동', '중계본동', '중림동', '하계1동']
seoul_cluster3 = ['상계10동', '상계3.4동', '상계5동', '상계8동', '월계2동', '월계3동', '청구동', '하계2동']
seoul_cluster4 = ['공릉1동', '공릉2동', '광희동', '다산동', '상계6.7동', '신당5동', '신당동', '약수동', '을지로동', '장충동', '황학동']

# 대구 cluster 1,2,3,4
daegu_cluster1 = ['대봉1동', '대신동', '두산동', '범어1동', '범어2동', '삼덕동', '성내1동', '성내2동', '중동']
daegu_cluster2 = ['고산1동', '고산3동', '남산4동', '만촌3동', '범물1동', '범어3동', '범어4동', '지산1동', '지산2동']
daegu_cluster3 = ['남산3동', '대봉2동', '범물2동', '상동', '수성1가동', '수성4가동', '파동', '황금1동']
daegu_cluster4 = ['고산2동', '남산1동', '남산2동', '동인동', '만촌1동', '만촌2동', '성내3동', '수성2.3가동', '황금2동']

# 서울 34개 동, 대구 35개 동
print(len(seoul_cluster1) + len(seoul_cluster2) + len(seoul_cluster3) + len(seoul_cluster4))
print(len(daegu_cluster1) + len(daegu_cluster2) + len(daegu_cluster3) + len(daegu_cluster4))

# 서울 , 대구 구분 없는 군집 리스트
cluster_1=['명동' ,'상계2동', '소공동' ,'필동', '회현동','대봉1동' ,'대신동', '두산동' ,'범어1동' ,'범어2동' ,'삼덕동', '성내1동', '성내2동' ,'중동']
cluster_2=['동화동' ,'상계1동', '상계9동' ,'월계1동', '중계1동' ,'중계2.3동', '중계4동' ,'중계본동', '중림동' ,'하계1동','고산1동' ,'고산3동' ,'남산4동', '만촌3동', '범물1동', '범어3동' ,'범어4동', '지산1동', '지산2동']
cluster_3=['상계10동', '상계3.4동' ,'상계5동', '상계8동' ,'월계2동' ,'월계3동' ,'청구동' ,'하계2동','남산3동' ,'대봉2동' ,'범물2동', '상동', '수성1가동' ,'수성4가동' ,'파동' ,'황금1동']
cluster_4=['공릉1동', '공릉2동' ,'광희동', '다산동', '상계6.7동', '신당5동', '신당동', '약수동', '을지로동' ,'장충동', '황학동','고산2동', '남산1동', '남산2동', '동인동', '만촌1동', '만촌2동', '성내3동', '수성2.3가동', '황금2동']
print(len(cluster_1), len(cluster_2), len(cluster_3), len(cluster_4))

34
35
14 19 16 20


In [20]:
boot_value = 0.3

# 부트스트랩 & 직접효과 제거 결과 read 해보기

# 부트스트랩 & 직접효과 제거 결과
boot_direct_mediation_result = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/boot_direct_mediation_result.csv', encoding='cp949')

# 부트스트랩 & 직접효과 제거 결과
card_weekday_list = boot_direct_mediation_result.iloc[0, ].values[0].split(',')
card_weekend_list = boot_direct_mediation_result.iloc[1, ].values[0].split(',')
gs_weekday_list = boot_direct_mediation_result.iloc[2, ].values[0].split(',')
gs_weekend_list = boot_direct_mediation_result.iloc[3, ].values[0].split(',')

print('카드 평일 부트스트랩, 직접효과 제거 : ', len(card_weekday_list), card_weekday_list)
print('카드 휴일 부트스트랩, 직접효과 제거 : ', len(card_weekend_list), card_weekend_list)
print('편의점 평일 부트스트랩, 직접효과 제거 : ', len(gs_weekday_list), gs_weekday_list)
print('편의점 휴일 부트스트랩, 직접효과 제거 : ', len(gs_weekend_list), gs_weekend_list)

카드 평일 부트스트랩, 직접효과 제거 :  44 ['상계10동', '수성4가동', '범어3동', '대봉2동', '중계4동', '수성1가동', '삼덕동', '회현동', '장충동', '황금2동', '대봉1동', '만촌2동', '범어2동', '수성2.3가동', '대신동', '만촌3동', '월계1동', '범어4동', '명동', '하계2동', '남산2동', '범어1동', '월계2동', '상계5동', '남산1동', '신당5동', '중계2.3동', '소공동', '남산3동', '범물1동', '남산4동', '상동', '두산동', '공릉2동', '하계1동', '중림동', '성내2동', '중계1동', '공릉1동', '을지로동', '성내1동', '상계1동', '청구동', '만촌1동']
카드 휴일 부트스트랩, 직접효과 제거 :  45 ['상계10동', '대봉2동', '중계4동', '상계6.7동', '장충동', '황금2동', '대봉1동', '만촌2동', '범어2동', '고산1동', '대신동', '만촌3동', '명동', '범어4동', '성내3동', '남산2동', '월계2동', '범어1동', '신당동', '상계5동', '고산2동', '남산1동', '중계2.3동', '신당5동', '필동', '소공동', '상계2동', '범물1동', '남산4동', '중계본동', '상동', '두산동', '동인동', '공릉2동', '황금1동', '하계1동', '고산3동', '중림동', '성내2동', '중동', '파동', '약수동', '을지로동', '상계1동', '만촌1동']
편의점 평일 부트스트랩, 직접효과 제거 :  45 ['상계10동', '수성4가동', '범어3동', '상계9동', '수성1가동', '삼덕동', '상계6.7동', '회현동', '장충동', '황금2동', '범어2동', '고산1동', '만촌3동', '월계1동', '범어4동', '상계1동', '성내3동', '범어1동', '신당동', '황학동', '상계5동', '고산2동', '남산1동', '신당5동', '필동', '소공동', '상계2동', '범물1동',

In [21]:
# 군집별 상관 유의 업종
# 카드 서울, 대구 평일, 휴일 seg 들 list - 16개 - AMT
card_seoul_weekday_seg1_corr40_list = ['레저용품(20)', '의복(42)', '보건위생(71)', '요식업소(80)']
card_seoul_weekday_seg2_corr40_list = ['요식업소(80)']
card_seoul_weekday_seg3_corr40_list = []
card_seoul_weekday_seg4_corr40_list = ['요식업소(80)']

card_seoul_weekend_seg1_corr40_list = ['레저용품(20)', '레저업소(21)', '유통업(40)', '의복(42)', '서적문구(50)', '요식업소(80)']
card_seoul_weekend_seg2_corr40_list = ['숙박(10)', '전기(31)', '유통업(40)', '보건위생(71)', '요식업소(80)']
card_seoul_weekend_seg3_corr40_list = ['요식업소(80)'] # 
card_seoul_weekend_seg4_corr40_list = ['보건위생(71)']

card_daegu_weekday_seg1_corr40_list = ['레저업소(21)', '연료판매(33)', '유통업(40)', '사무통신(52)', '의료기관(70)', '보건위생(71)', '요식업소(80)']
card_daegu_weekday_seg2_corr40_list = ['레저업소(21)', '의료기관(70)', '요식업소(80)']
card_daegu_weekday_seg3_corr40_list = ['레저업소(21)', '요식업소(80)']
card_daegu_weekday_seg4_corr40_list = ['레저업소(21)', '요식업소(80)']

card_daegu_weekend_seg1_corr40_list = ['문화취미(22)', '연료판매(33)', '유통업(40)', '서적문구(50)', '사무통신(52)', '요식업소(80)']
card_daegu_weekend_seg2_corr40_list = ['보건위생(71)', '요식업소(80)']
card_daegu_weekend_seg3_corr40_list = ['레저업소(21)', '사무통신(52)', '요식업소(80)'] # 
card_daegu_weekend_seg4_corr40_list = ['레저업소(21)', '연료판매(33)', '의복(42)', '요식업소(80)']

In [22]:
# 군집별 상관 유의 업종

# 편의점 서울, 대구 평일, 휴일 seg 들 list - 16개 
gs_seoul_weekday_seg1_corr40_list = ['마실거리']
gs_seoul_weekday_seg2_corr40_list = ['식사']
gs_seoul_weekday_seg3_corr40_list = []
gs_seoul_weekday_seg4_corr40_list = []

gs_seoul_weekend_seg1_corr40_list = ['취미&여가활동_비중','임신/육아_비중']
gs_seoul_weekend_seg2_corr40_list = ['식사','임신/육아_비중']#
gs_seoul_weekend_seg3_corr40_list = []
gs_seoul_weekend_seg4_corr40_list = ['마실거리','임신/육아_비중']

gs_daegu_weekday_seg1_corr40_list = ['식사', '마실거리', '홈&리빙']
gs_daegu_weekday_seg2_corr40_list = ['마실거리']
gs_daegu_weekday_seg3_corr40_list = []
gs_daegu_weekday_seg4_corr40_list = ['마실거리']

gs_daegu_weekend_seg1_corr40_list = ['간식', '마실거리', '홈&리빙', '헬스&뷰티', '임신/육아_비중']
gs_daegu_weekend_seg2_corr40_list = ['식사', '마실거리','임신/육아_비중']
gs_daegu_weekend_seg3_corr40_list = []
gs_daegu_weekend_seg4_corr40_list = ['마실거리']

## 모델링 하이퍼 파라미터

In [23]:
from sklearn.metrics import r2_score
# 모델링 p-value, r-squared 나타낼 때 소수점
float_list = 4
random_state_ = 77

## 카드 데이터 전처리
- 서울 평일 / 서울 휴일
- 대구 평일 / 대구 휴일



In [24]:
# card2020 년도 데이터
card2020 = card[card['STD_DD']>20191231]
card2020 = pd.merge(card2020, holiday, on='STD_DD')

# 평일, 휴일 나누기
card2020_weekday = card2020[card2020['weekend']==0]
card2020_weekend = card2020[card2020['weekend']==1]

print(card2020.shape, card2020_weekday.shape, card2020_weekend.shape)

# card2019 년도 데이터 
card2019 = card[card['STD_DD']<20191231]
card2019 = pd.merge(card2019, holiday, on='STD_DD')

# 2019년과 2020년을 날짜별(같은 주차의 같은 요일)로 맞추어주는 과정
card2019['STD_DD'] = card2019['STD_DD'].astype('str')
card2019['STD_DD'] = pd.to_datetime(card2019['STD_DD']) + timedelta(weeks=52)

# 2019년을 2020 년도 로 다 바꾸기
card2019['STD_DD'] = card2019['STD_DD'].astype(str)
card2019["STD_DD"] = card2019["STD_DD"].str.replace("-","")
card2019["STD_DD"] = card2019["STD_DD"].astype(int)

# 평일, 휴일 나누기
card2019_weekday = card2019[card2019['weekend']==0]
card2019_weekend = card2019[card2019['weekend']==1]

print(card2019.shape, card2019_weekday.shape, card2019_weekend.shape)

(1310849, 13) (906029, 13) (404820, 13)
(1465733, 13) (1008488, 13) (457245, 13)


### 2019, 2020 서울 카드 평일 처리

In [25]:
# 각 cluster와 서울 카드 평일 교집합 
seoul_weekday_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekday_list)))
seoul_weekday_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekday_list)))
seoul_weekday_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekday_list)))
seoul_weekday_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekday_list)))
print(seoul_weekday_seg1_list, '\n', seoul_weekday_seg2_list, '\n', seoul_weekday_seg3_list, '\n', seoul_weekday_seg4_list)
print('각 cluster와 서울 카드 평일 교집합 완료\n')

# 서울 평일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekday_seg1 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg1_list)]
card_seoul_weekday_seg2 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg2_list)] 
card_seoul_weekday_seg3 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg3_list)]
card_seoul_weekday_seg4 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg4_list)]

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)
print('서울 평일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 평일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg1 = pd.DataFrame(card_seoul_weekday_seg1)

card_seoul_weekday_seg2 = card_seoul_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg2 = pd.DataFrame(card_seoul_weekday_seg2)

card_seoul_weekday_seg3 = card_seoul_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg3 = pd.DataFrame(card_seoul_weekday_seg3)

card_seoul_weekday_seg4 = card_seoul_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg4 = pd.DataFrame(card_seoul_weekday_seg4)

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)
print('평일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')

# 데이터 reset_index
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.reset_index()
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.reset_index()
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.reset_index()
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekday_seg1["MCT_CAT_CD"] = card_seoul_weekday_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg2["MCT_CAT_CD"] = card_seoul_weekday_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg3["MCT_CAT_CD"] = card_seoul_weekday_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg4["MCT_CAT_CD"] = card_seoul_weekday_seg4["MCT_CAT_CD"].astype(str)
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

['명동', '회현동', '소공동'] 
 ['하계1동', '중림동', '상계1동', '월계1동', '중계4동', '중계2.3동', '중계1동'] 
 ['상계10동', '청구동', '하계2동', '월계2동', '상계5동'] 
 ['장충동', '공릉2동', '신당5동', '을지로동', '공릉1동']
각 cluster와 서울 카드 평일 교집합 완료

(100447, 13) (110571, 13) (51586, 13) (92661, 13)
서울 평일 각 seg 로 분리 데이터프레임 생성 완료

(1685, 1) (1746, 1) (1553, 1) (1852, 1)
평일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(1685, 3) (1746, 3) (1553, 3) (1852, 3)


In [26]:
# pivot table 로 펼침 (평일)
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (평일)')

# 결측치 0
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.fillna(0)
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.fillna(0)
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.fillna(0)
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.T
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.T
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.T
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.T

print('전치뒤집기 완료')
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekday_seg1_corr = pd.merge(holiday, card_seoul_weekday_seg1, on='STD_DD')
card_seoul_weekday_seg2_corr = pd.merge(holiday, card_seoul_weekday_seg2,on='STD_DD')
card_seoul_weekday_seg3_corr = pd.merge(holiday, card_seoul_weekday_seg3,on='STD_DD')
card_seoul_weekday_seg4_corr = pd.merge(holiday, card_seoul_weekday_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekday_seg1_corr_2019 = pd.merge(covid, card_seoul_weekday_seg1_corr, on='STD_DD')
card_seoul_weekday_seg2_corr_2019 = pd.merge(covid, card_seoul_weekday_seg2_corr,on='STD_DD')
card_seoul_weekday_seg3_corr_2019 = pd.merge(covid, card_seoul_weekday_seg3_corr,on='STD_DD')
card_seoul_weekday_seg4_corr_2019 = pd.merge(covid, card_seoul_weekday_seg4_corr,on='STD_DD')

print(card_seoul_weekday_seg1_corr_2019.shape , card_seoul_weekday_seg2_corr_2019.shape, card_seoul_weekday_seg3_corr_2019.shape, card_seoul_weekday_seg4_corr_2019.shape)

pivot table 로 펼침 (평일)
전치뒤집기 완료
(81, 22) (81, 23) (81, 23) (81, 23)
(80, 26) (80, 27) (80, 27) (80, 27)


In [27]:
# 각 cluster와 서울 카드 평일 교집합 
seoul_weekday_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekday_list)))
seoul_weekday_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekday_list)))
seoul_weekday_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekday_list)))
seoul_weekday_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekday_list)))
print(seoul_weekday_seg1_list, '\n', seoul_weekday_seg2_list, '\n', seoul_weekday_seg3_list, '\n', seoul_weekday_seg4_list)
print('각 cluster와 서울 카드 평일 교집합 완료\n')

# 서울 평일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekday_seg1 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg1_list)]
card_seoul_weekday_seg2 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg2_list)] 
card_seoul_weekday_seg3 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg3_list)]
card_seoul_weekday_seg4 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg4_list)]

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)
print('서울 평일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 평일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg1 = pd.DataFrame(card_seoul_weekday_seg1)

card_seoul_weekday_seg2 = card_seoul_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg2 = pd.DataFrame(card_seoul_weekday_seg2)

card_seoul_weekday_seg3 = card_seoul_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg3 = pd.DataFrame(card_seoul_weekday_seg3)

card_seoul_weekday_seg4 = card_seoul_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg4 = pd.DataFrame(card_seoul_weekday_seg4)

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)
print('평일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')

# 데이터 reset_index
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.reset_index()
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.reset_index()
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.reset_index()
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekday_seg1["MCT_CAT_CD"] = card_seoul_weekday_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg2["MCT_CAT_CD"] = card_seoul_weekday_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg3["MCT_CAT_CD"] = card_seoul_weekday_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg4["MCT_CAT_CD"] = card_seoul_weekday_seg4["MCT_CAT_CD"].astype(str)

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

['명동', '회현동', '소공동'] 
 ['하계1동', '중림동', '상계1동', '월계1동', '중계4동', '중계2.3동', '중계1동'] 
 ['상계10동', '청구동', '하계2동', '월계2동', '상계5동'] 
 ['장충동', '공릉2동', '신당5동', '을지로동', '공릉1동']
각 cluster와 서울 카드 평일 교집합 완료

(82565, 13) (103139, 13) (50125, 13) (84920, 13)
서울 평일 각 seg 로 분리 데이터프레임 생성 완료

(1674, 1) (1777, 1) (1605, 1) (1883, 1)
평일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(1674, 3) (1777, 3) (1605, 3) (1883, 3)


In [28]:
# pivot table 로 펼침 (평일)
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (평일)')

# 결측치 0
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.fillna(0)
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.fillna(0)
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.fillna(0)
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.T
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.T
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.T
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.T

print('전치뒤집기 완료')
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekday_seg1_corr = pd.merge(holiday, card_seoul_weekday_seg1, on='STD_DD')
card_seoul_weekday_seg2_corr = pd.merge(holiday, card_seoul_weekday_seg2,on='STD_DD')
card_seoul_weekday_seg3_corr = pd.merge(holiday, card_seoul_weekday_seg3,on='STD_DD')
card_seoul_weekday_seg4_corr = pd.merge(holiday, card_seoul_weekday_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekday_seg1_corr_2020 = pd.merge(covid, card_seoul_weekday_seg1_corr, on='STD_DD')
card_seoul_weekday_seg2_corr_2020 = pd.merge(covid, card_seoul_weekday_seg2_corr,on='STD_DD')
card_seoul_weekday_seg3_corr_2020 = pd.merge(covid, card_seoul_weekday_seg3_corr,on='STD_DD')
card_seoul_weekday_seg4_corr_2020 = pd.merge(covid, card_seoul_weekday_seg4_corr,on='STD_DD')

print(card_seoul_weekday_seg1_corr_2020.shape , card_seoul_weekday_seg2_corr_2020.shape, card_seoul_weekday_seg3_corr_2020.shape, card_seoul_weekday_seg4_corr_2020.shape)

pivot table 로 펼침 (평일)
전치뒤집기 완료
(82, 22) (82, 23) (82, 23) (82, 23)
(82, 26) (82, 27) (82, 27) (82, 27)


### 2019, 2020 서울 카드 휴일 처리

In [29]:
# 각 cluster와 서울 카드 휴일 교집합 
seoul_weekend_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekend_list)))
seoul_weekend_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekend_list)))
seoul_weekend_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekend_list)))
seoul_weekend_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekend_list)))
print(seoul_weekend_seg1_list, '\n', seoul_weekend_seg2_list, '\n', seoul_weekend_seg3_list, '\n', seoul_weekend_seg4_list)
print('각 cluster와 서울 카드 휴일 교집합 완료\n')

# 서울 휴일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekend_seg1 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg1_list)]
card_seoul_weekend_seg2 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg2_list)] 
card_seoul_weekend_seg3 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg3_list)]
card_seoul_weekend_seg4 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg4_list)]

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)
print('서울 휴일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 휴일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg1 = pd.DataFrame(card_seoul_weekend_seg1)

card_seoul_weekend_seg2 = card_seoul_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg2 = pd.DataFrame(card_seoul_weekend_seg2)

card_seoul_weekend_seg3 = card_seoul_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg3 = pd.DataFrame(card_seoul_weekend_seg3)

card_seoul_weekend_seg4 = card_seoul_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg4 = pd.DataFrame(card_seoul_weekend_seg4)

print('휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')
print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 데이터 reset_index
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.reset_index()
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.reset_index()
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.reset_index()
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekend_seg1["MCT_CAT_CD"] = card_seoul_weekend_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg2["MCT_CAT_CD"] = card_seoul_weekend_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg3["MCT_CAT_CD"] = card_seoul_weekend_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg4["MCT_CAT_CD"] = card_seoul_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

['명동', '필동', '상계2동', '소공동'] 
 ['하계1동', '중계본동', '중림동', '상계1동', '중계4동', '중계2.3동'] 
 ['상계10동', '월계2동', '상계5동'] 
 ['신당동', '장충동', '공릉2동', '신당5동', '을지로동', '약수동', '상계6.7동']
각 cluster와 서울 카드 휴일 교집합 완료

(47522, 13) (44584, 13) (14630, 13) (53519, 13)
서울 휴일 각 seg 로 분리 데이터프레임 생성 완료

휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(818, 1) (815, 1) (674, 1) (839, 1)
(818, 3) (815, 3) (674, 3) (839, 3)


In [30]:
# pivot table 로 펼침 (휴일)
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (휴일)')

# 결측치 0
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.fillna(0)
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.fillna(0)
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.fillna(0)
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.T
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.T
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.T
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.T

print('전치뒤집기 완료')
print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekend_seg1_corr = pd.merge(holiday, card_seoul_weekend_seg1, on='STD_DD')
card_seoul_weekend_seg2_corr = pd.merge(holiday, card_seoul_weekend_seg2,on='STD_DD')
card_seoul_weekend_seg3_corr = pd.merge(holiday, card_seoul_weekend_seg3,on='STD_DD')
card_seoul_weekend_seg4_corr = pd.merge(holiday, card_seoul_weekend_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekend_seg1_corr_2019 = pd.merge(covid, card_seoul_weekend_seg1_corr, on='STD_DD')
card_seoul_weekend_seg2_corr_2019 = pd.merge(covid, card_seoul_weekend_seg2_corr,on='STD_DD')
card_seoul_weekend_seg3_corr_2019 = pd.merge(covid, card_seoul_weekend_seg3_corr,on='STD_DD')
card_seoul_weekend_seg4_corr_2019 = pd.merge(covid, card_seoul_weekend_seg4_corr,on='STD_DD')

print(card_seoul_weekend_seg1_corr_2019.shape , card_seoul_weekend_seg2_corr_2019.shape, card_seoul_weekend_seg3_corr_2019.shape, card_seoul_weekend_seg4_corr_2019.shape)

pivot table 로 펼침 (휴일)
전치뒤집기 완료
(39, 22) (39, 23) (39, 23) (39, 23)
(39, 26) (39, 27) (39, 27) (39, 27)


In [31]:
# 각 cluster와 서울 카드 휴일 교집합 
seoul_weekend_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekend_list)))
seoul_weekend_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekend_list)))
seoul_weekend_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekend_list)))
seoul_weekend_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekend_list)))
print(seoul_weekend_seg1_list, '\n', seoul_weekend_seg2_list, '\n', seoul_weekend_seg3_list, '\n', seoul_weekend_seg4_list)
print('각 cluster와 서울 카드 휴일 교집합 완료\n')

# 서울 휴일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekend_seg1 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg1_list)]
card_seoul_weekend_seg2 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg2_list)] 
card_seoul_weekend_seg3 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg3_list)]
card_seoul_weekend_seg4 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg4_list)]

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)
print('서울 휴일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 휴일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg1 = pd.DataFrame(card_seoul_weekend_seg1)

card_seoul_weekend_seg2 = card_seoul_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg2 = pd.DataFrame(card_seoul_weekend_seg2)

card_seoul_weekend_seg3 = card_seoul_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg3 = pd.DataFrame(card_seoul_weekend_seg3)

card_seoul_weekend_seg4 = card_seoul_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg4 = pd.DataFrame(card_seoul_weekend_seg4)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)
print('휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')

# 데이터 reset_index
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.reset_index()
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.reset_index()
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.reset_index()
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekend_seg1["MCT_CAT_CD"] = card_seoul_weekend_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg2["MCT_CAT_CD"] = card_seoul_weekend_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg3["MCT_CAT_CD"] = card_seoul_weekend_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg4["MCT_CAT_CD"] = card_seoul_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

['명동', '필동', '상계2동', '소공동'] 
 ['하계1동', '중계본동', '중림동', '상계1동', '중계4동', '중계2.3동'] 
 ['상계10동', '월계2동', '상계5동'] 
 ['신당동', '장충동', '공릉2동', '신당5동', '을지로동', '약수동', '상계6.7동']
각 cluster와 서울 카드 휴일 교집합 완료

(39194, 13) (41707, 13) (14530, 13) (48580, 13)
서울 휴일 각 seg 로 분리 데이터프레임 생성 완료

(818, 1) (820, 1) (680, 1) (847, 1)
휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(818, 3) (820, 3) (680, 3) (847, 3)


In [32]:
# pivot table 로 펼침 (휴일)
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (휴일)')

# 결측치 0
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.fillna(0)
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.fillna(0)
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.fillna(0)
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.T
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.T
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.T
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.T

print('전치뒤집기 완료')
print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekend_seg1_corr = pd.merge(holiday, card_seoul_weekend_seg1, on='STD_DD')
card_seoul_weekend_seg2_corr = pd.merge(holiday, card_seoul_weekend_seg2,on='STD_DD')
card_seoul_weekend_seg3_corr = pd.merge(holiday, card_seoul_weekend_seg3,on='STD_DD')
card_seoul_weekend_seg4_corr = pd.merge(holiday, card_seoul_weekend_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekend_seg1_corr_2020 = pd.merge(covid, card_seoul_weekend_seg1_corr, on='STD_DD')
card_seoul_weekend_seg2_corr_2020 = pd.merge(covid, card_seoul_weekend_seg2_corr,on='STD_DD')
card_seoul_weekend_seg3_corr_2020 = pd.merge(covid, card_seoul_weekend_seg3_corr,on='STD_DD')
card_seoul_weekend_seg4_corr_2020 = pd.merge(covid, card_seoul_weekend_seg4_corr,on='STD_DD')

print(card_seoul_weekend_seg1_corr_2020.shape , card_seoul_weekend_seg2_corr_2020.shape, card_seoul_weekend_seg3_corr_2020.shape, card_seoul_weekend_seg4_corr_2020.shape)

pivot table 로 펼침 (휴일)
전치뒤집기 완료
(39, 23) (39, 23) (39, 22) (39, 23)
(39, 27) (39, 27) (39, 26) (39, 27)


### 서울 카드 평일, 휴일 데이터 합침

In [33]:
# 서울 평일 데이터정리
# 10_x, 20_x --> 2019년도
# 10_y, 20_y --> 2020년도
card_seoul_weekday_19_20_seg1 = pd.merge(card_seoul_weekday_seg1_corr_2019, card_seoul_weekday_seg1_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_seoul_weekday_19_20_seg2 = pd.merge(card_seoul_weekday_seg2_corr_2019, card_seoul_weekday_seg2_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_seoul_weekday_19_20_seg3 = pd.merge(card_seoul_weekday_seg3_corr_2019, card_seoul_weekday_seg3_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_seoul_weekday_19_20_seg4 = pd.merge(card_seoul_weekday_seg4_corr_2019, card_seoul_weekday_seg4_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])

# example
print(card_seoul_weekday_seg1_corr_2019.shape, card_seoul_weekday_seg2_corr_2019.shape, card_seoul_weekday_seg3_corr_2019.shape, card_seoul_weekday_seg4_corr_2019.shape)
print(card_seoul_weekday_seg1_corr_2020.shape, card_seoul_weekday_seg2_corr_2020.shape, card_seoul_weekday_seg3_corr_2020.shape, card_seoul_weekday_seg4_corr_2020.shape)

print(card_seoul_weekday_19_20_seg1.shape, card_seoul_weekday_19_20_seg2.shape, card_seoul_weekday_19_20_seg3.shape, card_seoul_weekday_19_20_seg4.shape)
card_seoul_weekday_19_20_seg1

(80, 26) (80, 27) (80, 27) (80, 27)
(82, 26) (82, 27) (82, 27) (82, 27)
(77, 48) (77, 50) (77, 50) (77, 50)


,STD_DD,confirmed,day,weekend,10_x,20_x,21_x,22_x,30_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,10_y,20_y,21_y,22_y,30_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200206,5,3,0,10408.833333,10123.067797,1227.690476,384.982143,94.00,569.875000,843.458333,295.222222,2224.805556,67208.607843,3741.891892,674.533333,1449.275362,602.014706,2069.234043,0.0,2510.240506,9971.511905,5220.696629,26822.800000,870.521277,187.764706,5804.691489,7093.897959,1701.659091,219.135593,0.0,412.692308,762.361111,1335.545455,1354.156250,29479.689320,1849.144928,285.200000,1154.611940,741.414286,4398.408163,0.0,1721.858974,11407.545455,4121.590909,24773.854369,802.548387,173.666667
1,20200207,3,4,0,10892.577320,11618.884615,1503.439024,378.619048,0.00,1110.111111,809.833333,1436.115385,2162.888889,80452.784314,4198.275000,324.777778,2408.589041,1143.114286,2334.630435,0.0,1936.822785,9639.317647,6010.648936,33316.424242,701.566667,738.466667,8017.703297,7968.560000,1976.853659,188.172414,0.0,336.333333,567.354839,924.200000,1549.538462,43154.911765,2050.608108,291.833333,1693.597222,473.958904,4161.944444,2210.0,1833.960526,10304.568182,6012.827586,28478.158416,854.608696,1059.952381
2,20200210,3,0,0,11303.969072,9624.771930,1781.244444,402.700000,75.00,1591.600000,800.321429,1851.333333,1842.074074,47097.500000,4082.197183,552.714286,1663.363636,595.041667,2401.833333,0.0,2568.467532,11701.622222,5334.193548,27642.465347,679.663043,324.812500,7037.505495,9388.588235,2144.425000,161.540000,0.0,667.000000,770.733333,798.375000,1696.781250,24806.705882,2049.328358,85.000000,1766.215385,425.957746,3900.480769,27.0,2458.068493,9842.722222,5715.174419,24502.650485,791.760870,189.352941
3,20200211,0,1,0,8976.170000,9285.851852,1592.575000,291.949153,93.00,831.571429,1002.200000,1326.500000,1551.933333,49994.127451,3742.383562,805.470588,2083.194444,687.455882,3245.130435,0.0,1963.051282,9173.152941,5178.182796,29002.921569,751.450549,237.461538,5687.000000,8033.180000,1247.300000,285.578947,0.0,597.777778,681.913043,1178.000000,2561.230769,34262.000000,2473.850746,494.142857,1696.045455,552.352941,3794.367347,0.0,1672.069444,9582.636364,5682.404494,26739.693069,812.890110,140.250000
4,20200212,1,2,0,8347.134021,10451.220000,1762.883721,349.015385,388.75,995.000000,851.296296,429.760000,934.090909,46813.058824,4071.126761,348.900000,1892.536232,616.197183,3344.104167,0.0,1784.240506,8873.813953,5175.075269,30241.823529,1550.953488,574.619048,5318.617978,7458.354167,996.166667,235.296296,0.0,846.200000,723.842105,3358.705882,1157.555556,33424.623762,3155.283333,112.166667,1125.446154,491.937500,3266.538462,0.0,1620.794521,7975.933333,3746.955056,25693.598039,760.689655,195.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,20200525,25,0,0,11789.635417,9240.714286,1622.386364,345.440000,994.50,531.214286,739.565217,2409.150000,2352.285714,27167.960784,3383.462500,303.153846,2295.753247,566.605634,9664.040000,0.0,3080.826667,29275.956522,5595.640449,25657.990000,760.406250,195.125000,1442.468750,13816.214286,1727.294118,328.000000,0.0,1791.166667,681.809524,1193.727273,1716.066667,28388.730337,3175.568966,764.764706,1453.568627,440.610169,4915.880000,0.0,448.557143,29158.468354,3924.876712,28297.430108,781.220779,484.692308
73,20200526,16,1,0,8771.371134,10690.578947,1663.088889,355.967742,0.00,647.000000,1046.692308,375.181818,1806.483871,50630.126214,4494.775000,479.750000,1707.240506,580.346667,7929.679245,0.0,1921.526316,48570.761364,6150.589474,28728.460784,906.572917,295.814815,2084.714286,10467.133333,1321.114286,225.312500,0.0,735.050000,1160.720000,203.384615,1902.590909,34266.936170,3245.721311,1039.750000,1324.375000,764.568966,6300.547619,0.0,467.820896,54853.057143,4171.742857,28112.701031,953.055556,239.42

In [34]:
# 서울 휴일 데이터 정리
# 10_x, 20_x --> 2019년도
# 10_y, 20_y --> 2020년도
card_seoul_weekend_19_20_seg1 = pd.merge(card_seoul_weekend_seg1_corr_2019, card_seoul_weekend_seg1_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_seoul_weekend_19_20_seg2 = pd.merge(card_seoul_weekend_seg2_corr_2019, card_seoul_weekend_seg2_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_seoul_weekend_19_20_seg3 = pd.merge(card_seoul_weekend_seg3_corr_2019, card_seoul_weekend_seg3_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_seoul_weekend_19_20_seg4 = pd.merge(card_seoul_weekend_seg4_corr_2019, card_seoul_weekend_seg4_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
print(card_seoul_weekend_19_20_seg1.shape, card_seoul_weekend_19_20_seg2.shape, card_seoul_weekend_19_20_seg3.shape, card_seoul_weekend_19_20_seg4.shape)

# example
card_seoul_weekend_19_20_seg1

(34, 49) (34, 50) (34, 49) (34, 50)


,STD_DD,confirmed,day,weekend,10_x,20_x,21_x,22_x,30_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,10_y,20_y,21_y,22_y,30_y,31,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200201,4,5,1,8474.348315,19408.076923,1219.836364,1195.355263,73.000000,516.714286,794.686275,818.440000,4119.555556,40948.467213,4713.164179,187.600000,1514.440678,1047.955882,364.391304,77.000000,707.883117,5790.574257,4490.198020,22884.590551,900.245283,243.312500,8490.659091,12363.076923,1103.898305,243.563380,0.000000,0.0,120.857143,537.208333,894.636364,5453.928571,32575.945312,2703.234375,393.000000,1319.468750,759.042857,190.043478,205.500000,498.750000,5834.902913,4636.863158,20852.147287,532.971963,197.285714
1,20200202,1,6,1,7980.494382,15551.307692,1033.840000,839.426471,0.000000,131.000000,573.818182,212.100000,1989.250000,30830.952381,3559.393443,242.000000,1347.862069,423.537037,272.050000,0.000000,350.656716,915.140351,3367.966292,19372.862903,712.164835,218.600000,7424.569767,13387.816327,1076.886792,258.016129,137.000000,0.0,101.571429,517.357143,132.764706,6155.000000,24059.222222,2582.428571,442.000000,1816.285714,471.098361,198.684211,0.000000,164.046154,1602.729412,3574.652632,16653.255814,568.961538,121.500000
2,20200208,0,5,1,10360.258427,17317.849057,1068.242424,821.053333,2060.000000,234.000000,706.333333,1440.083333,1056.636364,47521.642857,5009.446154,188.750000,2218.634921,1084.500000,314.200000,68.000000,703.935897,6375.574468,5669.831683,25165.929134,582.104762,198.260870,10577.324675,12218.274510,905.903226,327.300000,0.000000,0.0,135.142857,568.562500,941.043478,718.750000,10267.870968,2911.015873,124.400000,1696.873016,690.000000,274.076923,139.666667,586.430556,5418.372549,5071.940476,20312.658915,468.000000,377.846154
3,20200209,0,6,1,12288.488372,17135.288462,1038.472727,734.313433,0.000000,85.000000,303.714286,774.923077,3379.000000,44556.566667,4848.796610,67.000000,2086.984375,718.147541,159.950000,0.000000,415.385714,1106.030303,4729.061856,21130.344000,496.514851,168.625000,6434.525000,13375.470588,901.403846,305.916667,0.000000,0.0,110.750000,355.641026,1158.250000,15705.000000,10343.292683,2642.285714,256.000000,2461.462963,459.083333,202.000000,0.000000,268.420290,902.457143,4959.000000,17982.690476,336.841584,432.666667
4,20200215,0,5,1,10647.629213,16351.807692,890.276923,819.106667,4972.000000,270.750000,760.125000,435.689655,641.000000,41633.048387,4932.055556,80.000000,1878.382353,669.452055,161.172414,1963.600000,876.041667,5644.767677,5666.803922,26370.054264,596.931373,360.750000,5210.597561,15284.019231,1086.769231,565.101266,619.000000,0.0,322.888889,617.285714,1771.208333,2492.250000,29097.388889,3643.184615,413.300000,2672.417910,654.088235,219.125000,774.000000,280.430556,5114.525253,4835.000000,23450.793651,468.644860,230.105263
5,20200216,0,6,1,11103.883721,17501.169811,757.440678,972.611940,0.000000,142.500000,492.692308,266.866667,1875.500000,35602.365854,4906.852459,147.000000,1675.900000,538.166667,180.187500,0.000000,466.736111,883.785714,4705.242105,20396.923077,644.825688,261.090909,5760.684211,13578.360000,1285.781818,373.421875,0.000000,0.0,313.000000,368.309524,266.000000,9058.500000,24806.581967,1929.368421,950.000000,1379.547170,374.614035,224.529412,0.000000,147.271429,1273.236111,3294.931818,17311.582677,348.117647,188.750000
6,20200222,100,5,1,10892.589474,18485.054545,1100.850000,823.405405,94.500000,447.800000,764.122807,738.657143,1080.538462,46457.901639,5164.885714,106.142857,2484.507246,590.890411,432.952381,287.000000,790.126761,6819.357143,5303.778846,28418.056452,549.342593,182.517241,6951.908046,9709.920000,1240.490909,292.372881,1944.333333,0.0,100.222222,642.333333,559.500000,309.600000,23655.992248,2206.870968,40.000000,1791.265625,528.000000,995.318182,165.500000,262.073529,5450.690000,3926.117021,17925.629921,473.621359,243.263158
7,20200223,229,6,1,9465.897727,21050.622642,861.489796,7

### 2019, 2020 대구 카드 평일 처리

In [35]:
# 각 cluster와 서울 카드 평일 교집합 
daegu_weekday_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekday_list)))
daegu_weekday_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekday_list)))
daegu_weekday_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekday_list)))
daegu_weekday_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekday_list)))
print(daegu_weekday_seg1_list, '\n', daegu_weekday_seg2_list, '\n', daegu_weekday_seg3_list, '\n', daegu_weekday_seg4_list)
print('각 cluster와 서울 카드 평일 교집합 완료\n')

# 서울 평일 각 seg 로 분리 데이터프레임 생성
card_daegu_weekday_seg1 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg1_list)]
card_daegu_weekday_seg2 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg2_list)] 
card_daegu_weekday_seg3 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg3_list)]
card_daegu_weekday_seg4 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg4_list)]

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)
print('서울 평일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 평일 각 seg별 일별, 업종별 USE_AMT 평균
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg1 = pd.DataFrame(card_daegu_weekday_seg1)

card_daegu_weekday_seg2 = card_daegu_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg2 = pd.DataFrame(card_daegu_weekday_seg2)

card_daegu_weekday_seg3 = card_daegu_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg3 = pd.DataFrame(card_daegu_weekday_seg3)

card_daegu_weekday_seg4 = card_daegu_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg4 = pd.DataFrame(card_daegu_weekday_seg4)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)
print('평일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')

# 데이터 reset_index
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.reset_index()
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.reset_index()
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.reset_index()
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekday_seg1["MCT_CAT_CD"] = card_daegu_weekday_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg2["MCT_CAT_CD"] = card_daegu_weekday_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg3["MCT_CAT_CD"] = card_daegu_weekday_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg4["MCT_CAT_CD"] = card_daegu_weekday_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

['대신동', '성내1동', '대봉1동', '범어1동', '두산동', '삼덕동', '성내2동', '범어2동'] 
 ['남산4동', '범물1동', '범어3동', '범어4동', '만촌3동'] 
 ['수성4가동', '수성1가동', '대봉2동', '남산3동', '상동'] 
 ['황금2동', '수성2.3가동', '만촌1동', '남산2동', '남산1동', '만촌2동']
각 cluster와 서울 카드 평일 교집합 완료

(138836, 13) (54256, 13) (40593, 13) (66193, 13)
서울 평일 각 seg 로 분리 데이터프레임 생성 완료

(1822, 1) (1395, 1) (1431, 1) (1690, 1)
평일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(1822, 3) (1395, 3) (1431, 3) (1690, 3)


In [36]:
# pivot table 로 펼침 (평일)
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (평일)')

# 결측치 0
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.fillna(0)
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.fillna(0)
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.fillna(0)
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.T
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.T
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.T
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.T

print('전치뒤집기 완료')
print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekday_seg1_corr = pd.merge(holiday, card_daegu_weekday_seg1, on='STD_DD')
card_daegu_weekday_seg2_corr = pd.merge(holiday, card_daegu_weekday_seg2,on='STD_DD')
card_daegu_weekday_seg3_corr = pd.merge(holiday, card_daegu_weekday_seg3,on='STD_DD')
card_daegu_weekday_seg4_corr = pd.merge(holiday, card_daegu_weekday_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekday_seg1_corr_2019 = pd.merge(covid, card_daegu_weekday_seg1_corr, on='STD_DD')
card_daegu_weekday_seg2_corr_2019 = pd.merge(covid, card_daegu_weekday_seg2_corr,on='STD_DD')
card_daegu_weekday_seg3_corr_2019 = pd.merge(covid, card_daegu_weekday_seg3_corr,on='STD_DD')
card_daegu_weekday_seg4_corr_2019 = pd.merge(covid, card_daegu_weekday_seg4_corr,on='STD_DD')

print(card_daegu_weekday_seg1_corr_2019.shape , card_daegu_weekday_seg2_corr_2019.shape, card_daegu_weekday_seg3_corr_2019.shape, card_daegu_weekday_seg4_corr_2019.shape)

pivot table 로 펼침 (평일)
전치뒤집기 완료
(81, 23) (81, 22) (81, 23) (81, 23)
(80, 27) (80, 26) (80, 27) (80, 27)


In [37]:
# 각 cluster와 서울 카드 평일 교집합 
daegu_weekday_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekday_list)))
daegu_weekday_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekday_list)))
daegu_weekday_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekday_list)))
daegu_weekday_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekday_list)))
print(daegu_weekday_seg1_list, '\n', daegu_weekday_seg2_list, '\n', daegu_weekday_seg3_list, '\n', daegu_weekday_seg4_list)
print('각 cluster와 서울 카드 평일 교집합 완료\n')

# 서울 평일 각 seg 로 분리 데이터프레임 생성
card_daegu_weekday_seg1 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg1_list)]
card_daegu_weekday_seg2 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg2_list)] 
card_daegu_weekday_seg3 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg3_list)]
card_daegu_weekday_seg4 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg4_list)]

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)
print('서울 평일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 평일 각 seg별 일별, 업종별 USE_AMT 평균
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg1 = pd.DataFrame(card_daegu_weekday_seg1)

card_daegu_weekday_seg2 = card_daegu_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg2 = pd.DataFrame(card_daegu_weekday_seg2)

card_daegu_weekday_seg3 = card_daegu_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg3 = pd.DataFrame(card_daegu_weekday_seg3)

card_daegu_weekday_seg4 = card_daegu_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg4 = pd.DataFrame(card_daegu_weekday_seg4)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)
print('평일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')

# 데이터 reset_index
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.reset_index()
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.reset_index()
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.reset_index()
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekday_seg1["MCT_CAT_CD"] = card_daegu_weekday_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg2["MCT_CAT_CD"] = card_daegu_weekday_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg3["MCT_CAT_CD"] = card_daegu_weekday_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg4["MCT_CAT_CD"] = card_daegu_weekday_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

['대신동', '성내1동', '대봉1동', '범어1동', '두산동', '삼덕동', '성내2동', '범어2동'] 
 ['남산4동', '범물1동', '범어3동', '범어4동', '만촌3동'] 
 ['수성4가동', '수성1가동', '대봉2동', '남산3동', '상동'] 
 ['황금2동', '수성2.3가동', '만촌1동', '남산2동', '남산1동', '만촌2동']
각 cluster와 서울 카드 평일 교집합 완료

(116703, 13) (48941, 13) (37527, 13) (59204, 13)
서울 평일 각 seg 로 분리 데이터프레임 생성 완료

(1816, 1) (1407, 1) (1422, 1) (1653, 1)
평일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(1816, 3) (1407, 3) (1422, 3) (1653, 3)


In [38]:
# pivot table 로 펼침 (평일)
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (평일)')

# 결측치 0
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.fillna(0)
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.fillna(0)
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.fillna(0)
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.T
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.T
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.T
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.T

print('전치뒤집기 완료')
print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekday_seg1_corr = pd.merge(holiday, card_daegu_weekday_seg1, on='STD_DD')
card_daegu_weekday_seg2_corr = pd.merge(holiday, card_daegu_weekday_seg2,on='STD_DD')
card_daegu_weekday_seg3_corr = pd.merge(holiday, card_daegu_weekday_seg3,on='STD_DD')
card_daegu_weekday_seg4_corr = pd.merge(holiday, card_daegu_weekday_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekday_seg1_corr_2020 = pd.merge(covid, card_daegu_weekday_seg1_corr, on='STD_DD')
card_daegu_weekday_seg2_corr_2020 = pd.merge(covid, card_daegu_weekday_seg2_corr,on='STD_DD')
card_daegu_weekday_seg3_corr_2020 = pd.merge(covid, card_daegu_weekday_seg3_corr,on='STD_DD')
card_daegu_weekday_seg4_corr_2020 = pd.merge(covid, card_daegu_weekday_seg4_corr,on='STD_DD')

print(card_daegu_weekday_seg1_corr_2020.shape , card_daegu_weekday_seg2_corr_2020.shape, card_daegu_weekday_seg3_corr_2020.shape, card_daegu_weekday_seg4_corr_2020.shape)

pivot table 로 펼침 (평일)
전치뒤집기 완료
(82, 23) (82, 22) (82, 23) (82, 23)
(82, 27) (82, 26) (82, 27) (82, 27)


### 2019, 2020 대구 카드 휴일 처리

In [39]:
# 각 cluster와 서울 카드 휴일 교집합 
daegu_weekend_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekend_list)))
daegu_weekend_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekend_list)))
daegu_weekend_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekend_list)))
daegu_weekend_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekend_list)))
print(daegu_weekend_seg1_list, '\n', daegu_weekend_seg2_list, '\n', daegu_weekend_seg3_list, '\n', daegu_weekend_seg4_list)
print('각 cluster와 서울 카드 휴일 교집합 완료\n')

# 서울 휴일 각 seg 로 분리 데이터프레임 생성
card_daegu_weekend_seg1 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg1_list)]
card_daegu_weekend_seg2 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg2_list)] 
card_daegu_weekend_seg3 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg3_list)]
card_daegu_weekend_seg4 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg4_list)]

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)
print('서울 휴일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 휴일 각 seg별 일별, 업종별 USE_AMT 평균
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg1 = pd.DataFrame(card_daegu_weekend_seg1)

card_daegu_weekend_seg2 = card_daegu_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg2 = pd.DataFrame(card_daegu_weekend_seg2)

card_daegu_weekend_seg3 = card_daegu_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg3 = pd.DataFrame(card_daegu_weekend_seg3)

card_daegu_weekend_seg4 = card_daegu_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg4 = pd.DataFrame(card_daegu_weekend_seg4)

print('휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')
print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 데이터 reset_index
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.reset_index()
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.reset_index()
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.reset_index()
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekend_seg1["MCT_CAT_CD"] = card_daegu_weekend_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg2["MCT_CAT_CD"] = card_daegu_weekend_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg3["MCT_CAT_CD"] = card_daegu_weekend_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg4["MCT_CAT_CD"] = card_daegu_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

['중동', '대신동', '대봉1동', '범어1동', '두산동', '성내2동', '범어2동'] 
 ['남산4동', '고산3동', '범물1동', '범어4동', '만촌3동', '고산1동'] 
 ['대봉2동', '파동', '상동', '황금1동'] 
 ['황금2동', '만촌1동', '성내3동', '남산2동', '남산1동', '동인동', '고산2동', '만촌2동']
각 cluster와 서울 카드 휴일 교집합 완료

(49571, 13) (33214, 13) (13897, 13) (39272, 13)
서울 휴일 각 seg 로 분리 데이터프레임 생성 완료

휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(848, 1) (686, 1) (611, 1) (836, 1)
(848, 3) (686, 3) (611, 3) (836, 3)


In [40]:
# pivot table 로 펼침 (휴일)
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (휴일)')

# 결측치 0
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.fillna(0)
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.fillna(0)
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.fillna(0)
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.T
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.T
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.T
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.T

print('전치뒤집기 완료')
print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekend_seg1_corr = pd.merge(holiday, card_daegu_weekend_seg1, on='STD_DD')
card_daegu_weekend_seg2_corr = pd.merge(holiday, card_daegu_weekend_seg2,on='STD_DD')
card_daegu_weekend_seg3_corr = pd.merge(holiday, card_daegu_weekend_seg3,on='STD_DD')
card_daegu_weekend_seg4_corr = pd.merge(holiday, card_daegu_weekend_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekend_seg1_corr_2019 = pd.merge(covid, card_daegu_weekend_seg1_corr, on='STD_DD')
card_daegu_weekend_seg2_corr_2019 = pd.merge(covid, card_daegu_weekend_seg2_corr,on='STD_DD')
card_daegu_weekend_seg3_corr_2019 = pd.merge(covid, card_daegu_weekend_seg3_corr,on='STD_DD')
card_daegu_weekend_seg4_corr_2019 = pd.merge(covid, card_daegu_weekend_seg4_corr,on='STD_DD')

print(card_daegu_weekend_seg1_corr_2019.shape , card_daegu_weekend_seg2_corr_2019.shape, card_daegu_weekend_seg3_corr_2019.shape, card_daegu_weekend_seg4_corr_2019.shape)

pivot table 로 펼침 (휴일)
전치뒤집기 완료
(39, 23) (39, 22) (39, 22) (39, 23)
(39, 27) (39, 26) (39, 26) (39, 27)


In [41]:
# 각 cluster와 서울 카드 휴일 교집합 
daegu_weekend_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekend_list)))
daegu_weekend_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekend_list)))
daegu_weekend_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekend_list)))
daegu_weekend_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekend_list)))
print(daegu_weekend_seg1_list, '\n', daegu_weekend_seg2_list, '\n', daegu_weekend_seg3_list, '\n', daegu_weekend_seg4_list)
print('각 cluster와 서울 카드 휴일 교집합 완료\n')

# 서울 휴일 각 seg 로 분리 데이터프레임 생성
card_daegu_weekend_seg1 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg1_list)]
card_daegu_weekend_seg2 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg2_list)] 
card_daegu_weekend_seg3 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg3_list)]
card_daegu_weekend_seg4 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg4_list)]

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)
print('서울 휴일 각 seg 로 분리 데이터프레임 생성 완료\n')

# 휴일 각 seg별 일별, 업종별 USE_AMT 평균
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg1 = pd.DataFrame(card_daegu_weekend_seg1)

card_daegu_weekend_seg2 = card_daegu_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg2 = pd.DataFrame(card_daegu_weekend_seg2)

card_daegu_weekend_seg3 = card_daegu_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg3 = pd.DataFrame(card_daegu_weekend_seg3)

card_daegu_weekend_seg4 = card_daegu_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg4 = pd.DataFrame(card_daegu_weekend_seg4)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)
print('휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료\n')

# 데이터 reset_index
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.reset_index()
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.reset_index()
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.reset_index()
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekend_seg1["MCT_CAT_CD"] = card_daegu_weekend_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg2["MCT_CAT_CD"] = card_daegu_weekend_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg3["MCT_CAT_CD"] = card_daegu_weekend_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg4["MCT_CAT_CD"] = card_daegu_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

['중동', '대신동', '대봉1동', '범어1동', '두산동', '성내2동', '범어2동'] 
 ['남산4동', '고산3동', '범물1동', '범어4동', '만촌3동', '고산1동'] 
 ['대봉2동', '파동', '상동', '황금1동'] 
 ['황금2동', '만촌1동', '성내3동', '남산2동', '남산1동', '동인동', '고산2동', '만촌2동']
각 cluster와 서울 카드 휴일 교집합 완료

(41824, 13) (29425, 13) (12234, 13) (34492, 13)
서울 휴일 각 seg 로 분리 데이터프레임 생성 완료

(836, 1) (701, 1) (590, 1) (818, 1)
휴일 각 seg별 일별, 업종별 USE_AMT 평균 완료

(836, 3) (701, 3) (590, 3) (818, 3)


In [42]:
# pivot table 로 펼침 (휴일)
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
print('pivot table 로 펼침 (휴일)')

# 결측치 0
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.fillna(0)
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.fillna(0)
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.fillna(0)
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.T
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.T
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.T
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.T

print('전치뒤집기 완료')
print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekend_seg1_corr = pd.merge(holiday, card_daegu_weekend_seg1, on='STD_DD')
card_daegu_weekend_seg2_corr = pd.merge(holiday, card_daegu_weekend_seg2,on='STD_DD')
card_daegu_weekend_seg3_corr = pd.merge(holiday, card_daegu_weekend_seg3,on='STD_DD')
card_daegu_weekend_seg4_corr = pd.merge(holiday, card_daegu_weekend_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekend_seg1_corr_2020 = pd.merge(covid, card_daegu_weekend_seg1_corr, on='STD_DD')
card_daegu_weekend_seg2_corr_2020 = pd.merge(covid, card_daegu_weekend_seg2_corr,on='STD_DD')
card_daegu_weekend_seg3_corr_2020 = pd.merge(covid, card_daegu_weekend_seg3_corr,on='STD_DD')
card_daegu_weekend_seg4_corr_2020 = pd.merge(covid, card_daegu_weekend_seg4_corr,on='STD_DD')

print(card_daegu_weekend_seg1_corr_2020.shape , card_daegu_weekend_seg2_corr_2020.shape, card_daegu_weekend_seg3_corr_2020.shape, card_daegu_weekend_seg4_corr_2020.shape)

pivot table 로 펼침 (휴일)
전치뒤집기 완료
(39, 23) (39, 23) (39, 22) (39, 23)
(39, 27) (39, 27) (39, 26) (39, 27)


### 대구 카드 평일, 휴일 데이터 합침

In [43]:
# 대구 평일 데이터정리
# 10_x, 20_x --> 2019년도
# 10_y, 20_y --> 2020년도
card_daegu_weekday_19_20_seg1 = pd.merge(card_daegu_weekday_seg1_corr_2019, card_daegu_weekday_seg1_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_daegu_weekday_19_20_seg2 = pd.merge(card_daegu_weekday_seg2_corr_2019, card_daegu_weekday_seg2_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_daegu_weekday_19_20_seg3 = pd.merge(card_daegu_weekday_seg3_corr_2019, card_daegu_weekday_seg3_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_daegu_weekday_19_20_seg4 = pd.merge(card_daegu_weekday_seg4_corr_2019, card_daegu_weekday_seg4_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])

# example
print(card_daegu_weekday_19_20_seg1.shape, card_daegu_weekday_19_20_seg2.shape, card_daegu_weekday_19_20_seg3.shape, card_daegu_weekday_19_20_seg4.shape)
card_daegu_weekday_19_20_seg1

(77, 50) (77, 48) (77, 50) (77, 50)


,STD_DD,confirmed,day,weekend,10_x,20_x,21_x,22_x,30_x,31_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,10_y,20_y,21_y,22_y,30_y,31_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200206,5,3,0,815.701754,1792.037736,668.848837,669.365591,2322.500000,81.0,485.500000,1443.530120,826.882353,3725.727273,6956.500000,2813.408602,646.000000,1926.709091,656.029412,7121.157895,1867.833333,309.629213,10231.339394,1214.686567,5416.309278,404.006667,125.888889,693.329114,664.772727,483.370787,613.904762,0.000000,511.0,242.500000,1334.142857,354.576923,1300.400000,6242.899471,1899.988889,626.083333,1824.867925,357.383333,7272.000000,1432.000000,159.250000,7446.529762,1021.741071,4342.964286,431.937888,149.454545
1,20200207,3,4,0,914.060606,1731.411765,757.533333,832.564356,3300.250000,900.0,2782.400000,1462.076923,746.225806,2162.916667,6217.546392,3228.530000,1287.941176,1334.509091,567.485294,8209.075472,2270.750000,266.103774,10559.588957,1161.669173,6573.331633,682.794702,214.789474,789.909091,877.909091,624.038462,506.380282,5518.000000,0.0,583.166667,1387.416667,587.958333,1501.250000,6277.943590,2288.712871,1253.400000,691.485294,450.492754,8581.780000,2056.428571,177.415730,10802.830409,933.543860,5664.281915,523.951220,143.291667
2,20200210,3,0,0,997.486486,1250.552632,932.022727,745.063830,3967.333333,81.0,776.625000,1452.439024,1094.560000,3221.857143,5358.196891,2312.931373,2143.277778,1244.754386,472.660714,7810.380000,6010.400000,370.750000,8789.327381,949.763359,4581.256545,579.006329,165.937500,901.986111,1349.693878,566.730000,653.573529,5004.666667,0.0,697.454545,1596.444444,1357.080000,3151.250000,2612.902174,1888.484536,1992.000000,1195.796296,421.718750,6739.301887,1046.200000,278.047619,8868.245614,1051.864000,4208.915842,482.257862,138.555556
3,20200211,0,1,0,1137.294118,1317.586957,789.000000,774.776596,5410.000000,0.0,410.200000,1562.569620,254.500000,1060.909091,5908.898990,2450.460784,1823.187500,1042.344828,505.895522,8244.717391,898.666667,145.305263,10431.335404,1113.207207,4666.894472,503.811321,130.375000,829.120000,1104.160000,654.258427,347.720000,3478.000000,754.0,1245.000000,1321.457831,166.947368,1932.444444,6613.768041,2070.030000,3002.916667,1110.655738,396.295082,6845.535714,1618.571429,210.721519,9817.459627,915.983051,4640.170984,486.221519,200.423077
4,20200212,1,2,0,1010.623377,1595.418605,1035.458824,795.308511,3354.000000,525.0,283.900000,1500.261905,458.960000,2318.846154,7269.143590,2367.295238,836.153846,903.000000,622.630769,8114.220000,1309.750000,202.258427,9000.569620,1325.560000,5296.854167,547.358974,168.642857,611.013333,924.477273,761.000000,428.058824,4295.000000,0.0,363.666667,1315.607595,199.777778,871.111111,6851.178010,1858.152174,1009.384615,987.060000,578.400000,7679.882353,2351.400000,272.793103,8758.806250,677.758065,4482.331606,333.198718,106.227273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,20200525,25,0,0,2087.575000,1210.179487,653.859155,1110.822222,3076.000000,0.0,635.666667,1571.414634,619.160000,1968.500000,3772.171123,1860.725490,1042.950000,1127.250000,370.015625,7842.666667,629.500000,335.698925,9582.579268,977.937008,3999.722513,416.936709,268.120000,644.434783,1288.660377,913.882353,352.152542,6864.500000,4721.0,1361.636364,1323.302632,144.368421,4909.200000,4999.276836,2363.290000,837.263158,955.530303,748.533333,6774.285714,2632.000000,328.837209,9278.071856,1123.352941,3725.873626,629.696203,284.838710
73,20200526,16,1,0,1378.883117,906.022222,685.148148,573.891304,585.333333,5720.0,832.500000,1570.282353,194.700000,567.800000,6000.129353,2869.034188,748.105263,1494.753425,485.000000,8112.196078,499.500000,319.325843,9065.801205,998.370079,4162.577114,458.536585,179.827586,801.076923,1155.880952,855.913580,401.04761

In [44]:
# 대구 휴일 데이터 정리
# 10_x, 20_x --> 2019년도
# 10_y, 20_y --> 2020년도
card_daegu_weekend_19_20_seg1 = pd.merge(card_daegu_weekend_seg1_corr_2019, card_daegu_weekend_seg1_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_daegu_weekend_19_20_seg2 = pd.merge(card_daegu_weekend_seg2_corr_2019, card_daegu_weekend_seg2_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_daegu_weekend_19_20_seg3 = pd.merge(card_daegu_weekend_seg3_corr_2019, card_daegu_weekend_seg3_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
card_daegu_weekend_19_20_seg4 = pd.merge(card_daegu_weekend_seg4_corr_2019, card_daegu_weekend_seg4_corr_2020, on=['STD_DD', 'confirmed','day','weekend'])
print(card_daegu_weekend_19_20_seg1.shape, card_daegu_weekend_19_20_seg2.shape, card_daegu_weekend_19_20_seg3.shape, card_daegu_weekend_19_20_seg4.shape)

# example
card_daegu_weekend_19_20_seg1

(34, 50) (34, 49) (34, 48) (34, 50)


,STD_DD,confirmed,day,weekend,10_x,20_x,21_x,22_x,30_x,31_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,10_y,20_y,21_y,22_y,30_y,31_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200201,4,5,1,781.671875,1982.947368,704.597403,892.706667,8147.666667,0.0,1819.529412,2572.336538,643.312500,8602.600000,12908.439490,2264.100000,717.692308,624.305556,360.461538,14021.150000,1231.142857,1032.413333,4623.535714,1105.137615,6369.613924,1236.085714,168.066667,1140.307692,570.296296,623.320988,486.782609,6288.666667,0.0,792.166667,2312.846154,1523.272727,1676.875000,8130.709091,1840.075758,656.923077,678.000000,428.857143,13046.527778,3591.333333,324.942857,3724.866667,781.520833,5107.000000,498.753521,131.384615
1,20200202,1,6,1,779.848485,1297.842105,487.957746,928.241935,0.000000,0.0,613.857143,2294.980198,763.250000,5375.076923,12607.364780,1385.020000,1301.307692,241.117647,201.368421,17342.400000,0.000000,76.295455,1002.791667,798.397849,5916.493976,1080.692857,58.000000,1457.972222,668.958333,399.368421,399.192982,5006.500000,0.0,458.666667,1977.000000,746.647059,3327.700000,7805.920732,1069.512195,246.000000,334.200000,268.240000,11438.218750,5270.000000,174.225000,735.285714,638.457831,3542.317365,474.720000,117.500000
2,20200208,0,5,1,875.222222,1012.608696,680.514706,874.625000,4165.750000,7029.0,1321.571429,2276.895238,1372.166667,3549.909091,10431.950311,2264.441176,1263.117647,503.166667,377.040000,10912.522727,9485.000000,411.160494,4767.985185,844.930000,5433.677019,591.237037,98.850000,1446.835616,2132.409091,768.975000,530.233333,10011.000000,0.0,805.571429,2209.070000,1237.466667,3042.714286,9477.689441,2039.870370,1266.000000,678.897436,585.054054,11532.268293,896.000000,364.281250,4000.817518,689.437500,5009.451807,518.939850,77.952381
3,20200209,0,6,1,858.210526,4598.400000,675.090909,969.534483,1282.000000,0.0,590.375000,2162.257732,1598.076923,5516.500000,8003.183544,2728.377049,1188.214286,1659.000000,308.153846,11906.354839,29755.000000,147.940000,1323.281250,737.250000,4218.896341,392.279070,108.000000,2873.308824,1826.391304,479.246575,350.979167,6186.000000,0.0,916.333333,2086.303030,1137.125000,3815.625000,7637.446541,1309.018519,743.400000,1460.500000,213.750000,13118.517241,4391.666667,127.136364,1010.887097,629.056818,3983.666667,397.173228,26.000000
4,20200215,0,5,1,1488.187500,735.411765,880.737500,883.315068,399.250000,0.0,1477.111111,2521.852941,1197.631579,5313.416667,11908.477987,2288.596774,1722.368421,789.974359,603.307692,13162.307692,1560.833333,389.637681,4358.022059,776.362745,5961.628931,663.466165,88.920000,2017.549296,1456.391304,765.573171,360.854545,6069.333333,0.0,2521.250000,2369.556701,1702.722222,1879.166667,9783.223602,2066.876923,1345.066667,993.891892,329.000000,14134.405405,9431.500000,322.253521,3914.613139,779.590909,6003.935673,776.292857,189.125000
5,20200216,0,6,1,945.048780,1718.636364,343.200000,762.774194,5778.666667,0.0,120.250000,2304.489362,1225.578947,5635.545455,9797.124224,805.789474,549.571429,186.142857,269.650000,12822.812500,0.000000,231.604651,831.785714,594.430380,5253.296774,398.155039,108.000000,1834.055556,1088.666667,323.907895,606.285714,4582.000000,0.0,153.000000,2027.135922,889.307692,2179.000000,9581.870370,948.403846,587.333333,1342.391304,344.608696,11997.800000,4321.400000,109.527778,708.810345,602.133333,4083.500000,395.950820,125.250000
6,20200222,100,5,1,2649.794872,892.875000,405.227273,923.305556,5203.750000,3273.0,710.588235,2382.590476,709.555556,4185.333333,11078.500000,2726.842857,1397.857143,732.578947,427.552632,14007.410256,5926.600000,538.948052,3937.528986,1023.350515,6732.522013,606.572464,132.166667,1405.166667,435.750000,335.312500,575.928571,3749.000000,377.0,142.000000,1403.206522,1512.600000,1537.125000,4230.083871,1227.045455,116.000000,386.000000,183.000000,4781.454545,1628.500000,347.555556,3205.242424,499.072727,1244.993103,491.167939,149.2941

### 카드 날씨데이터 합침

In [45]:
# card 서울 / card 대구 / pvalue 결과 리스트
card_seoul_pvalue_test = []
card_daegu_pvalue_test = []

# 서울 카드 평일 + 서울 날씨 데이터
card_seoul_weekday_19_20_seg1 = pd.merge(card_seoul_weekday_19_20_seg1, seoul_weather, on='STD_DD')
card_seoul_weekday_19_20_seg2 = pd.merge(card_seoul_weekday_19_20_seg2, seoul_weather, on='STD_DD')
card_seoul_weekday_19_20_seg3 = pd.merge(card_seoul_weekday_19_20_seg3, seoul_weather, on='STD_DD')
card_seoul_weekday_19_20_seg4 = pd.merge(card_seoul_weekday_19_20_seg4, seoul_weather, on='STD_DD')

# 서울 카드 휴일
card_seoul_weekend_19_20_seg1 = pd.merge(card_seoul_weekend_19_20_seg1, seoul_weather, on='STD_DD')
card_seoul_weekend_19_20_seg2 = pd.merge(card_seoul_weekend_19_20_seg2, seoul_weather, on='STD_DD')
card_seoul_weekend_19_20_seg3 = pd.merge(card_seoul_weekend_19_20_seg3, seoul_weather, on='STD_DD')
card_seoul_weekend_19_20_seg4 = pd.merge(card_seoul_weekend_19_20_seg4, seoul_weather, on='STD_DD')

# 대구 카드 평일
card_daegu_weekday_19_20_seg1 = pd.merge(card_daegu_weekday_19_20_seg1, daegu_weather, on='STD_DD')
card_daegu_weekday_19_20_seg2 = pd.merge(card_daegu_weekday_19_20_seg2, daegu_weather, on='STD_DD')
card_daegu_weekday_19_20_seg3 = pd.merge(card_daegu_weekday_19_20_seg3, daegu_weather, on='STD_DD')
card_daegu_weekday_19_20_seg4 = pd.merge(card_daegu_weekday_19_20_seg4, daegu_weather, on='STD_DD')

# 대구 카드 휴일
card_daegu_weekend_19_20_seg1 = pd.merge(card_daegu_weekend_19_20_seg1, daegu_weather, on='STD_DD')
card_daegu_weekend_19_20_seg2 = pd.merge(card_daegu_weekend_19_20_seg2, daegu_weather, on='STD_DD')
card_daegu_weekend_19_20_seg3 = pd.merge(card_daegu_weekend_19_20_seg3, daegu_weather, on='STD_DD')
card_daegu_weekend_19_20_seg4 = pd.merge(card_daegu_weekend_19_20_seg4, daegu_weather, on='STD_DD')

# example
card_seoul_weekday_19_20_seg1

,STD_DD,confirmed,day,weekend,10_x,20_x,21_x,22_x,30_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,10_y,20_y,21_y,22_y,30_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y,temp_mean,rain_mean,hum_mean
0,20200206,5,3,0,10408.833333,10123.067797,1227.690476,384.982143,94.00,569.875000,843.458333,295.222222,2224.805556,67208.607843,3741.891892,674.533333,1449.275362,602.014706,2069.234043,0.0,2510.240506,9971.511905,5220.696629,26822.800000,870.521277,187.764706,5804.691489,7093.897959,1701.659091,219.135593,0.0,412.692308,762.361111,1335.545455,1354.156250,29479.689320,1849.144928,285.200000,1154.611940,741.414286,4398.408163,0.0,1721.858974,11407.545455,4121.590909,24773.854369,802.548387,173.666667,-6.4,0.0,36.4
1,20200207,3,4,0,10892.577320,11618.884615,1503.439024,378.619048,0.00,1110.111111,809.833333,1436.115385,2162.888889,80452.784314,4198.275000,324.777778,2408.589041,1143.114286,2334.630435,0.0,1936.822785,9639.317647,6010.648936,33316.424242,701.566667,738.466667,8017.703297,7968.560000,1976.853659,188.172414,0.0,336.333333,567.354839,924.200000,1549.538462,43154.911765,2050.608108,291.833333,1693.597222,473.958904,4161.944444,2210.0,1833.960526,10304.568182,6012.827586,28478.158416,854.608696,1059.952381,-1.7,0.0,45.1
2,20200210,3,0,0,11303.969072,9624.771930,1781.244444,402.700000,75.00,1591.600000,800.321429,1851.333333,1842.074074,47097.500000,4082.197183,552.714286,1663.363636,595.041667,2401.833333,0.0,2568.467532,11701.622222,5334.193548,27642.465347,679.663043,324.812500,7037.505495,9388.588235,2144.425000,161.540000,0.0,667.000000,770.733333,798.375000,1696.781250,24806.705882,2049.328358,85.000000,1766.215385,425.957746,3900.480769,27.0,2458.068493,9842.722222,5715.174419,24502.650485,791.760870,189.352941,4.0,0.0,49.4
3,20200211,0,1,0,8976.170000,9285.851852,1592.575000,291.949153,93.00,831.571429,1002.200000,1326.500000,1551.933333,49994.127451,3742.383562,805.470588,2083.194444,687.455882,3245.130435,0.0,1963.051282,9173.152941,5178.182796,29002.921569,751.450549,237.461538,5687.000000,8033.180000,1247.300000,285.578947,0.0,597.777778,681.913043,1178.000000,2561.230769,34262.000000,2473.850746,494.142857,1696.045455,552.352941,3794.367347,0.0,1672.069444,9582.636364,5682.404494,26739.693069,812.890110,140.250000,6.8,0.0,37.0
4,20200212,1,2,0,8347.134021,10451.220000,1762.883721,349.015385,388.75,995.000000,851.296296,429.760000,934.090909,46813.058824,4071.126761,348.900000,1892.536232,616.197183,3344.104167,0.0,1784.240506,8873.813953,5175.075269,30241.823529,1550.953488,574.619048,5318.617978,7458.354167,996.166667,235.296296,0.0,846.200000,723.842105,3358.705882,1157.555556,33424.623762,3155.283333,112.166667,1125.446154,491.937500,3266.538462,0.0,1620.794521,7975.933333,3746.955056,25693.598039,760.689655,195.000000,7.4,6.7,73.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,20200525,25,0,0,11789.635417,9240.714286,1622.386364,345.440000,994.50,531.214286,739.565217,2409.150000,2352.285714,27167.960784,3383.462500,303.153846,2295.753247,566.605634,9664.040000,0.0,3080.826667,29275.956522,5595.640449,25657.990000,760.406250,195.125000,1442.468750,13816.214286,1727.294118,328.000000,0.0,1791.166667,681.809524,1193.727273,1716.066667,28388.730337,3175.568966,764.764706,1453.568627,440.610169,4915.880000,0.0,448.557143,29158.468354,3924.876712,28297.430108,781.220779,484.692308,16.6,0.0,81.0
73,20200526,16,1,0,8771.371134,10690.578947,1663.088889,355.967742,0.00,647.000000,1046.692308,375.181818,1806.483871,50630.126214,4494.775000,479.750000,1707.240506,580.346667,7929.679245,0.0,1921.526316,48570.761364,6150.589474,28728.460784,906.572917,295.814815,2084.714286,10467.133333,1321.114286,225.312500,0.0,735.050000,1160.720000,203.384615,1902.590909,34266.936170,3245.72

## 카드매출액 회귀모델

#### 서울 카드 평일 군집1-4

In [46]:
# 서울 카드 평일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도
model_list_cc3 = [] # 코로나, 코로나 3일

business_type = []
for i in card_seoul_weekday_seg1_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekday_19_20_seg1['confirmed'])
  X2 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekday_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X3)
  result4 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X6)
  result7 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekday_19_20_seg1[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 평일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], 
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울평일군집1-상권.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 평일 군집1-상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저용품(20),"([0.0, 0.0], 0.29, [9351.675, -9.6042])","([0.0, 0.0, 0.0], 0.531, [7004.0081, -6.5698, ...","([0.0, 0.0, 0.6691], 0.291, [9315.9152, -9.575...","([0.0, 0.0, 0.0098], 0.351, [7066.7671, -9.201...","([0.0, 0.0, 0.0, 0.7145], 0.532, [7008.7597, -...","([0.0, 0.0, 0.0, 0.0167], 0.567, [5365.3682, -...","([0.0, 0.0, 0.3655, 0.0072], 0.359, [6728.4223...","([0.0, 0.0, 0.0, 0.0874, 0.0037], 0.584, [4792..."
1,의복(42),"([0.0, 0.0], 0.234, [2362.9129, -2.5962])","([0.0, 0.0006, 0.0], 0.394, [1787.9908, -1.853...","([0.0, 0.0, 0.7791], 0.235, [2355.5752, -2.590...","([0.0, 0.0, 0.0053], 0.311, [1595.8509, -2.460...","([0.0, 0.0006, 0.0, 0.7466], 0.395, [1789.4266...","([0.0, 0.0005, 0.0001, 0.0104], 0.446, [1189.5...","([0.0, 0.0, 0.2323, 0.0027], 0.324, [1457.6264...","([0.001, 0.0006, 0.0, 0.079, 0.0021], 0.47, [9..."
2,보건위생(71),"([0.0, 0.0006], 0.146, [3622.018, -3.6037])","([0.0, 0.0007, 0.568], 0.15, [3777.4197, -3.80...","([0.0, 0.0007, 0.8867], 0.146, [3628.9327, -3....","([0.0, 0.0009, 0.0462], 0.191, [2590.3156, -3....","([0.0, 0.0007, 0.5806, 0.952], 0.15, [3777.976...","([0.0, 0.0007, 0.4013, 0.0381], 0.199, [2761.8...","([0.0001, 0.0009, 0.2126, 0.0195], 0.208, [231...","([0.0001, 0.0008, 0.4699, 0.2436, 0.018], 0.21..."
3,요식업소(80),"([0.0, 0.0], 0.246, [26459.2545, -10.5716])","([0.0, 0.0002, 0.0045], 0.325, [24863.0315, -8...","([0.0, 0.0, 0.3678], 0.255, [26551.7199, -10.6...","([0.0, 0.0, 0.1481], 0.267, [24861.6322, -10.2...","([0.0, 0.0002, 0.0026, 0.166], 0.342, [24888.6...","([0.0, 0.0002, 0.0072, 0.2454], 0.337, [23707....","([0.0, 0.0, 0.0685, 0.0323], 0.3, [24004.9, -1...","([0.0, 0.0002, 0.0032, 0.028, 0.0389], 0.38, [..."


In [47]:
# 서울 카드 평일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekday_seg2_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekday_19_20_seg2['confirmed'])
  X2 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekday_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X3)
  result4 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X6)
  result7 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekday_19_20_seg2[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 평일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울평일군집2-주거상권.csv',index=False, encoding='cp949')

pvalue_test

서울 카드 평일 군집2-주거상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,요식업소(80),"([0.0, 0.0006], 0.147, [3434.3642, -1.571])","([0.0, 0.0183, 0.0001], 0.299, [3007.1431, -1....","([0.0, 0.0007, 0.6069], 0.15, [3423.5365, -1.5...","([0.0, 0.0009, 0.0318], 0.199, [2953.1917, -1....","([0.0, 0.0191, 0.0002, 0.992], 0.299, [3007.17...","([0.0, 0.0206, 0.0003, 0.0628], 0.331, [2646.8...","([0.0, 0.0009, 0.5731, 0.0319], 0.202, [2899.3...","([0.0, 0.0227, 0.0002, 0.3218, 0.036], 0.34, [..."


In [48]:
# 서울 카드 평일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekday_seg3_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekday_19_20_seg3['confirmed'])
  X2 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekday_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X3)
  result4 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X6)
  result7 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekday_19_20_seg3[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 평일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울평일군집3-주거.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 평일 군집3-주거


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh


In [49]:
# 서울 카드 평일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekday_seg4_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekday_19_20_seg4['confirmed'])
  X2 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekday_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X3)
  result4 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X6)
  result7 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekday_19_20_seg4[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 평일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울평일군집4-교통중심.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 평일 군집4-교통중심


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,요식업소(80),"([0.0, 0.0], 0.245, [5042.0134, -2.9369])","([0.0, 0.0004, 0.0], 0.416, [4385.7562, -2.088...","([0.0, 0.0, 0.7786], 0.246, [5050.0688, -2.943...","([0.0, 0.0, 0.1053], 0.272, [4544.7632, -2.849...","([0.0, 0.0004, 0.0, 0.3183], 0.424, [4390.5447...","([0.0, 0.0004, 0.0, 0.2083], 0.429, [4062.1179...","([0.0, 0.0, 0.2311, 0.047], 0.286, [4387.5193,...","([0.0, 0.0004, 0.0, 0.0715, 0.0504], 0.454, [3..."


#### 서울 카드 휴일 군집1-4

In [50]:
# 서울 카드 휴일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekend_seg1_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekend_19_20_seg1['confirmed'])
  X2 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekend_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X3)
  result4 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X6)
  result7 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekend_19_20_seg1[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 휴일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울휴일군집1-상권.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 휴일 군집1-상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저용품(20),"([0.0, 0.0021], 0.259, [16110.3308, -12.8034])","([0.0, 0.0015, 0.0], 0.648, [10993.4502, -9.55...","([0.0, 0.0034, 0.754], 0.261, [15988.874, -12....","([0.0001, 0.0035, 0.0114], 0.399, [10161.2379,...","([0.0, 0.0021, 0.0, 0.9135], 0.648, [10969.645...","([0.0, 0.0019, 0.0, 0.2002], 0.667, [9095.7234...","([0.0012, 0.0023, 0.2134, 0.0057], 0.43, [8946...","([0.0001, 0.0018, 0.0001, 0.4757, 0.149], 0.67..."
1,레저업소(21),"([0.0, 0.0316], 0.136, [848.3921, -0.3672])","([0.0, 0.0067, 0.0153], 0.288, [974.5056, -0.4...","([0.0, 0.0401, 0.8738], 0.137, [845.7645, -0.3...","([0.0, 0.0527, 0.1397], 0.196, [695.234, -0.32...","([0.0, 0.0097, 0.0163, 0.7481], 0.29, [970.553...","([0.0, 0.005, 0.0004, 0.0026], 0.477, [738.345...","([0.0, 0.0467, 0.4647, 0.1055], 0.21, [662.325...","([0.0, 0.0021, 0.0001, 0.086, 0.0006], 0.528, ..."
2,유통업(40),"([0.0, 0.0263], 0.145, [22893.9446, -8.907])","([0.0, 0.0554, 0.0733], 0.23, [20666.2239, -7....","([0.0, 0.0387, 0.5911], 0.153, [22686.3694, -8...","([0.0, 0.0459, 0.0695], 0.233, [18527.5782, -7...","([0.0, 0.0733, 0.0819, 0.6528], 0.236, [20530....","([0.0, 0.0685, 0.2327, 0.2195], 0.269, [18163....","([0.0, 0.0444, 0.6275, 0.0763], 0.239, [18024....","([0.0, 0.0703, 0.2698, 0.7922, 0.2487], 0.27, ..."
3,의복(42),"([0.0, 0.0001], 0.381, [2546.8667, -2.1591])","([0.0, 0.0002, 0.0169], 0.487, [2176.5473, -1....","([0.0, 0.0001, 0.3163], 0.401, [2595.8227, -2....","([0.0, 0.0002, 0.1077], 0.432, [2051.8651, -2....","([0.0, 0.0001, 0.0137, 0.216], 0.513, [2221.45...","([0.0, 0.0003, 0.0564, 0.4292], 0.497, [1978.3...","([0.0, 0.0, 0.0212, 0.0089], 0.525, [1757.1737...","([0.0, 0.0001, 0.1104, 0.0413, 0.0698], 0.566,..."
4,서적문구(50),"([0.0, 0.0002], 0.347, [593.9312, -0.5193])","([0.0, 0.0005, 0.016], 0.46, [497.322, -0.458,...","([0.0, 0.0004, 0.9638], 0.347, [594.5091, -0.5...","([0.0, 0.0004, 0.0207], 0.452, [413.6914, -0.4...","([0.0, 0.0007, 0.0174, 0.844], 0.461, [499.191...","([0.0, 0.0007, 0.0891, 0.1172], 0.503, [397.39...","([0.0001, 0.0002, 0.1171, 0.0057], 0.496, [362...","([0.0001, 0.0004, 0.1472, 0.1948, 0.0451], 0.5..."
5,요식업소(80),"([0.0, 0.0016], 0.27, [20042.1492, -8.1293])","([0.0, 0.003, 0.0011], 0.485, [17674.4732, -6....","([0.0, 0.0023, 0.9843], 0.27, [20037.4264, -8....","([0.0, 0.003, 0.0609], 0.349, [17258.2384, -7....","([0.0, 0.0037, 0.0013, 0.8529], 0.486, [17705....","([0.0, 0.004, 0.006, 0.4193], 0.496, [16766.95...","([0.0, 0.0021, 0.2429, 0.0291], 0.379, [16518....","([0.0, 0.0036, 0.0105, 0.4531, 0.2815], 0.506,..."


In [51]:
# 서울 카드 휴일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekend_seg2_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekend_19_20_seg2['confirmed'])
  X2 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekend_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X3)
  result4 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X6)
  result7 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekend_19_20_seg2[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 휴일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울휴일군집2-주거상권.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 휴일 군집2-주거상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,숙박(10),"([0.0, 0.0271], 0.144, [215.5583, 0.1756])","([0.0, 0.025, 0.5869], 0.152, [201.8391, 0.184...","([0.0, 0.0493, 0.1792], 0.193, [225.7206, 0.15...","([0.0002, 0.0317, 0.9988], 0.144, [215.4827, 0...","([0.0, 0.0432, 0.5275, 0.1728], 0.204, [210.24...","([0.0003, 0.0293, 0.5605, 0.8176], 0.153, [211...","([0.0022, 0.0447, 0.1176, 0.4012], 0.212, [182...","([0.0026, 0.0461, 0.7664, 0.1447, 0.5358], 0.2..."
1,전기(31),"([0.4413, 0.0002], 0.348, [-26.1121, 0.628])","([0.4725, 0.0003, 0.7229], 0.351, [-44.0703, 0...","([0.4383, 0.0003, 0.8376], 0.349, [-29.2626, 0...","([0.0702, 0.0001, 0.098], 0.404, [-184.8596, 0...","([0.4662, 0.0004, 0.7356, 0.8559], 0.352, [-46...","([0.0828, 0.0002, 0.7456, 0.1048], 0.406, [-18...","([0.0514, 0.0002, 0.4297, 0.0723], 0.417, [-21...","([0.0571, 0.0003, 0.6293, 0.3916, 0.0717], 0.4..."
2,유통업(40),"([0.0, 0.039], 0.126, [7913.8756, -2.9026])","([0.0, 0.0769, 0.0], 0.591, [6099.449, -1.7509...","([0.0, 0.0531, 0.6945], 0.131, [7860.2883, -2....","([0.0, 0.0687, 0.0308], 0.25, [6101.3248, -2.4...","([0.0, 0.093, 0.0, 0.8249], 0.592, [6082.5134,...","([0.0, 0.0916, 0.0, 0.4522], 0.599, [5706.8051...","([0.0, 0.0575, 0.3752, 0.0232], 0.27, [5785.60...","([0.0, 0.0943, 0.0, 0.8208, 0.4583], 0.6, [564..."
3,보건위생(71),"([0.0, 0.0278], 0.142, [713.8299, -0.3844])","([0.0, 0.0588, 0.0667], 0.232, [614.489, -0.32...","([0.0, 0.0338, 0.9724], 0.142, [713.2451, -0.3...","([0.0, 0.0482, 0.0793], 0.225, [529.3918, -0.3...","([0.0, 0.0652, 0.0711, 0.941], 0.232, [615.458...","([0.0, 0.0724, 0.2073, 0.2512], 0.265, [512.58...","([0.0003, 0.0372, 0.2858, 0.0424], 0.254, [481...","([0.0003, 0.0594, 0.2833, 0.3975, 0.1591], 0.2..."
4,요식업소(80),"([0.0, 0.0012], 0.284, [4609.4916, -2.3566])","([0.0, 0.0016, 0.0002], 0.551, [3864.5342, -1....","([0.0, 0.0021, 0.5788], 0.291, [4572.38, -2.28...","([0.0, 0.0016, 0.0043], 0.452, [3466.9708, -2....","([0.0, 0.0026, 0.0002, 0.6592], 0.554, [3845.3...","([0.0, 0.0018, 0.0022, 0.0637], 0.6, [3327.380...","([0.0, 0.0012, 0.2522, 0.0029], 0.475, [3278.8...","([0.0, 0.0017, 0.0041, 0.4898, 0.0574], 0.607,..."


In [52]:
# 서울 카드 휴일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekend_seg3_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekend_19_20_seg3['confirmed'])
  X2 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekend_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X3)
  result4 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X6)
  result7 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekend_19_20_seg3[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 휴일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울휴일군집3-주거.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 휴일 군집3-주거


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,요식업소(80),"([0.0, 0.0251], 0.147, [1145.0311, -0.3746])","([0.0, 0.058, 0.0011], 0.399, [985.3534, -0.27...","([0.0, 0.0237, 0.6101], 0.154, [1153.2463, -0....","([0.0, 0.0419, 0.1617], 0.2, [1003.199, -0.337...","([0.0, 0.047, 0.0011, 0.4177], 0.412, [994.305...","([0.0, 0.0658, 0.0036, 0.8144], 0.4, [967.4857...","([0.0, 0.026, 0.1322, 0.048], 0.259, [937.8157...","([0.0, 0.0476, 0.0071, 0.2654, 0.4168], 0.426,..."


In [53]:
# 서울 카드 휴일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_seoul_weekend_seg4_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_seoul_weekend_19_20_seg4['confirmed'])
  X2 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_seoul_weekend_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X1) 
  result2 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X2) 
  result3 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X3)
  result4 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X4) 
  result5 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X5) 
  result6 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X6)
  result7 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X7) 
  result8 = sm.OLS(card_seoul_weekend_19_20_seg4[x2020], X8)
  
  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 카드 휴일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드서울휴일군집4-교통중심.csv',index=False, encoding='cp949')
pvalue_test

서울 카드 휴일 군집4-교통중심


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,보건위생(71),"([0.0, 0.0398], 0.126, [828.1142, -0.3526])","([0.0, 0.0853, 0.0373], 0.241, [717.7102, -0.2...","([0.0, 0.0377, 0.6306], 0.132, [836.1129, -0.3...","([0.0, 0.0621, 0.2497], 0.163, [707.0637, -0.3...","([0.0, 0.0748, 0.0365, 0.5249], 0.252, [725.77...","([0.0, 0.0974, 0.0801, 0.6958], 0.245, [683.66...","([0.0, 0.0437, 0.1947, 0.0979], 0.209, [648.02...","([0.0, 0.0748, 0.1248, 0.3123, 0.3777], 0.272,..."


#### 대구 카드 평일 군집1-4

In [54]:
# 대구 카드 평일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekday_seg1_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekday_19_20_seg1['confirmed'])
  X2 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekday_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X3)
  result4 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X6)
  result7 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekday_19_20_seg1[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 평일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구평일군집1-상권.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 평일 군집1-상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저업소(21),"([0.0, 0.0], 0.278, [647.6977, -0.8776])","([0.0, 0.0, 0.0072], 0.346, [502.0967, -0.7262...","([0.0, 0.0, 0.6965], 0.279, [644.6285, -0.8751...","([0.0, 0.0, 0.8344], 0.278, [630.33, -0.8821, ...","([0.0, 0.0, 0.0083, 0.9477], 0.346, [502.037, ...","([0.0, 0.0001, 0.0078, 0.9793], 0.346, [504.05...","([0.0, 0.0, 0.7436, 0.9958], 0.279, [644.1369,...","([0.0, 0.0001, 0.0087, 0.9268, 0.9444], 0.346,..."
1,연료판매(33),"([0.0, 0.0], 0.288, [1185.722, -0.6474])","([0.0, 0.0, 0.7505], 0.289, [1198.424, -0.6606...","([0.0, 0.0, 0.784], 0.289, [1187.2753, -0.6487...","([0.0, 0.0, 0.1589], 0.307, [1269.415, -0.6255...","([0.0, 0.0, 0.7763, 0.8143], 0.289, [1198.5855...","([0.0, 0.0, 0.8452, 0.1685], 0.307, [1276.1555...","([0.0, 0.0, 0.586, 0.1398], 0.31, [1285.7361, ...","([0.0, 0.0, 0.8063, 0.5746, 0.1444], 0.31, [12..."
2,유통업(40),"([0.0, 0.0], 0.303, [5489.4472, -5.095])","([0.0, 0.0, 0.0028], 0.383, [4608.3082, -4.178...","([0.0, 0.0, 0.7345], 0.304, [5474.8494, -5.083...","([0.0, 0.0, 0.8339], 0.303, [5394.167, -5.1199...","([0.0, 0.0, 0.0032, 0.9802], 0.383, [4608.4303...","([0.0, 0.0, 0.003, 0.9585], 0.383, [4629.4432,...","([0.0, 0.0, 0.7892, 0.9704], 0.304, [5455.8922...","([0.0, 0.0, 0.0034, 0.998, 0.9637], 0.383, [46..."
3,사무통신(52),"([0.0, 0.0], 0.447, [6413.2015, -7.4477])","([0.0, 0.0, 0.216], 0.459, [6013.2312, -7.0318...","([0.0, 0.0, 0.4282], 0.452, [6376.7629, -7.417...","([0.0, 0.0, 0.7945], 0.448, [6286.5311, -7.480...","([0.0, 0.0, 0.253, 0.5188], 0.462, [6009.6622,...","([0.0, 0.0, 0.226, 0.8793], 0.459, [5943.7451,...","([0.0, 0.0, 0.4444, 0.8565], 0.452, [6475.1991...","([0.0, 0.0, 0.2539, 0.5104, 0.8311], 0.462, [6..."
4,의료기관(70),"([0.0, 0.0], 0.416, [7864.3731, -7.7266])","([0.0, 0.0, 0.8556], 0.417, [7799.0092, -7.658...","([0.0, 0.0, 0.4525], 0.421, [7826.1812, -7.695...","([0.0, 0.0, 0.5223], 0.419, [7520.9136, -7.816...","([0.0, 0.0, 0.9292, 0.4658], 0.421, [7794.5068...","([0.0, 0.0, 0.9005, 0.534], 0.42, [7481.6891, ...","([0.0, 0.0, 0.6274, 0.7736], 0.421, [7653.0952...","([0.0, 0.0, 0.936, 0.6365, 0.7769], 0.421, [76..."
5,보건위생(71),"([0.0, 0.0], 0.34, [894.611, -0.9197])","([0.0, 0.0, 0.0001], 0.458, [712.1625, -0.73, ...","([0.0, 0.0, 0.5392], 0.343, [890.2328, -0.9161...","([0.0, 0.0, 0.9382], 0.34, [900.4497, -0.9182,...","([0.0, 0.0, 0.0002, 0.8591], 0.458, [712.0229,...","([0.0, 0.0, 0.0001, 0.6592], 0.459, [740.7757,...","([0.0, 0.0, 0.4447, 0.6403], 0.345, [929.6717,...","([0.0, 0.0, 0.0002, 0.6358, 0.5349], 0.461, [7..."
6,요식업소(80),"([0.0, 0.0], 0.412, [3735.2091, -6.1192])","([0.0, 0.0, 0.0256], 0.451, [3105.6965, -5.464...","([0.0, 0.0, 0.6791], 0.414, [3718.3959, -6.105...","([0.0, 0.0, 0.5323], 0.415, [3467.5304, -6.189...","([0.0, 0.0, 0.029, 0.8853], 0.451, [3105.0076,...","([0.0, 0.0, 0.0297, 0.6587], 0.452, [2931.5747...","([0.0, 0.0, 0.9197, 0.6344], 0.415, [3489.4749...","([0.0, 0.0, 0.0308, 0.9211, 0.6699], 0.452, [2..."


In [55]:
# 대구 카드 평일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekday_seg2_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekday_19_20_seg2['confirmed'])
  X2 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekday_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X3)
  result4 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X6)
  result7 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekday_19_20_seg2[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 평일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구평일군집2-주거상권.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 평일 군집2-주거상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저업소(21),"([0.0, 0.0002], 0.166, [357.7159, -0.6695])","([0.0, 0.0007, 0.7816], 0.167, [341.3993, -0.6...","([0.0, 0.0003, 0.0864], 0.199, [343.5449, -0.6...","([0.0042, 0.0002, 0.3209], 0.177, [270.5544, -...","([0.0, 0.0006, 0.9469, 0.0926], 0.199, [339.71...","([0.0157, 0.0005, 0.8495, 0.3343], 0.178, [260...","([0.0016, 0.0003, 0.1635, 0.9064], 0.199, [332...","([0.0058, 0.0007, 0.9498, 0.17, 0.9085], 0.199..."
1,의료기관(70),"([0.0, 0.0], 0.285, [1712.7102, -1.6847])","([0.0, 0.0, 0.0854], 0.313, [1534.1266, -1.499...","([0.0, 0.0, 0.5703], 0.288, [1704.2772, -1.677...","([0.0, 0.0, 0.3447], 0.293, [1564.8459, -1.723...","([0.0, 0.0, 0.0984, 0.7172], 0.314, [1533.4858...","([0.0, 0.0, 0.1012, 0.4203], 0.319, [1416.5457...","([0.0, 0.0, 0.9319, 0.4501], 0.293, [1571.6188...","([0.0, 0.0, 0.1037, 0.948, 0.4728], 0.319, [14..."
2,요식업소(80),"([0.0, 0.0], 0.342, [1390.51, -1.63])","([0.0, 0.0, 0.0058], 0.407, [1151.9721, -1.381...","([0.0, 0.0, 0.3332], 0.351, [1378.3761, -1.620...","([0.0, 0.0, 0.89], 0.342, [1372.1456, -1.6348,...","([0.0, 0.0, 0.008, 0.5068], 0.411, [1151.0104,...","([0.0, 0.0, 0.0062, 0.9152], 0.407, [1164.7366...","([0.0, 0.0, 0.2962, 0.6658], 0.352, [1442.3827...","([0.0, 0.0, 0.008, 0.4026, 0.5987], 0.413, [12..."


In [56]:
# 대구 카드 평일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekday_seg3_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekday_19_20_seg3['confirmed'])
  X2 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekday_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X3)
  result4 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X6)
  result7 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekday_19_20_seg3[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 평일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구평일군집3-주거.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 평일 군집3-주거


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저업소(21),"([0.0, 0.0001], 0.181, [459.097, -0.8648])","([0.0, 0.001, 0.2011], 0.199, [367.409, -0.769...","([0.0, 0.0001, 0.8333], 0.182, [456.9445, -0.8...","([0.0, 0.0002, 0.1607], 0.203, [609.4832, -0.8...","([0.0, 0.001, 0.21, 0.9582], 0.199, [367.3446,...","([0.0001, 0.0021, 0.1555, 0.1256], 0.225, [521...","([0.0, 0.0004, 0.2636, 0.0761], 0.217, [669.75...","([0.0001, 0.0028, 0.1856, 0.3183, 0.069], 0.23..."
1,요식업소(80),"([0.0, 0.0], 0.329, [1234.9679, -1.5136])","([0.0, 0.0, 0.0038], 0.402, [996.3252, -1.2655...","([0.0, 0.0, 0.617], 0.332, [1228.9589, -1.5087...","([0.0, 0.0, 0.5396], 0.333, [1157.2253, -1.534...","([0.0, 0.0, 0.0046, 0.8751], 0.402, [996.1084,...","([0.0, 0.0, 0.0046, 0.7025], 0.403, [952.8144,...","([0.0, 0.0, 0.8348, 0.682], 0.333, [1170.6458,...","([0.0, 0.0, 0.005, 0.9617, 0.727], 0.403, [949..."


In [57]:
# 대구 카드 평일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekday_seg4_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekday_19_20_seg4['confirmed'])
  X2 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekday_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X3)
  result4 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X6)
  result7 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekday_19_20_seg4[x2020], X8)


  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 평일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구평일군집4-교통중심.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 평일 군집4-교통중심


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저업소(21),"([0.0, 0.0], 0.235, [388.6432, -0.6681])","([0.0, 0.0001, 0.1041], 0.262, [312.3642, -0.5...","([0.0, 0.0, 0.8018], 0.235, [386.9572, -0.6668...","([0.0, 0.0, 0.8544], 0.235, [375.6416, -0.6715...","([0.0, 0.0001, 0.1106, 0.9594], 0.262, [312.32...","([0.0004, 0.0002, 0.1087, 0.9677], 0.262, [309...","([0.0, 0.0, 0.8566, 0.9519], 0.235, [382.1496,...","([0.0013, 0.0002, 0.1133, 0.9723, 0.987], 0.26..."
1,요식업소(80),"([0.0, 0.0], 0.369, [1201.2759, -1.5775])","([0.0, 0.0, 0.0309], 0.408, [1029.3646, -1.398...","([0.0, 0.0, 0.4202], 0.375, [1192.0433, -1.569...","([0.0, 0.0, 0.4743], 0.374, [1114.7477, -1.600...","([0.0, 0.0, 0.039, 0.5771], 0.41, [1028.6122, ...","([0.0, 0.0, 0.0364, 0.5886], 0.41, [969.0842, ...","([0.0, 0.0, 0.6147, 0.7318], 0.376, [1145.6123...","([0.0, 0.0, 0.0411, 0.7475, 0.7701], 0.411, [9..."


#### 대구 카드 휴일 군집1-4

In [58]:
# 대구 카드 휴일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekend_seg1_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekend_19_20_seg1['confirmed'])
  X2 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekend_19_20_seg1[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X3)
  result4 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X6)
  result7 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekend_19_20_seg1[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))
  
# 최종 Data Frame
print('대구 카드 휴일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구휴일군집1-상권.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 휴일 군집1-상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,문화취미(22),"([0.0, 0.0351], 0.131, [308.3718, -0.2853])","([0.0, 0.0169, 0.1776], 0.182, [380.0403, -0.3...","([0.0, 0.0608, 0.2304], 0.171, [292.8039, -0.2...","([0.0503, 0.0238, 0.3182], 0.159, [208.6051, -...","([0.0, 0.0262, 0.1126, 0.1425], 0.239, [373.80...","([0.0223, 0.0047, 0.0551, 0.0894], 0.258, [238...","([0.0363, 0.0538, 0.3862, 0.5696], 0.18, [234....","([0.0156, 0.012, 0.0549, 0.3549, 0.2107], 0.28..."
1,연료판매(33),"([0.0, 0.0002], 0.358, [1782.0491, -1.111])","([0.0, 0.0002, 0.4484], 0.37, [1864.6978, -1.1...","([0.0, 0.0003, 0.8579], 0.358, [1777.2753, -1....","([0.0, 0.0003, 0.905], 0.358, [1757.643, -1.11...","([0.0, 0.0003, 0.432, 0.7609], 0.372, [1861.97...","([0.0, 0.0003, 0.3893, 0.6556], 0.374, [1785.8...","([0.0, 0.0007, 0.8879, 0.9591], 0.358, [1766.2...","([0.0, 0.0006, 0.3966, 0.8824, 0.7242], 0.374,..."
2,유통업(40),"([0.0, 0.0], 0.439, [7993.2394, -5.8685])","([0.0, 0.0001, 0.2638], 0.461, [7453.0663, -5....","([0.0, 0.0, 0.0364], 0.514, [7753.176, -5.4008...","([0.0, 0.0, 0.0218], 0.528, [5988.3423, -6.502...","([0.0, 0.0001, 0.4028, 0.0535], 0.525, [7379.0...","([0.0, 0.0, 0.7707, 0.0464], 0.529, [5949.0158...","([0.0, 0.0, 0.1754, 0.1009], 0.556, [6322.0089...","([0.0, 0.0001, 0.7768, 0.1832, 0.1569], 0.557,..."
3,서적문구(50),"([0.0, 0.0352], 0.131, [277.0055, -0.2364])","([0.0, 0.0179, 0.1981], 0.177, [333.7965, -0.2...","([0.0, 0.0467, 0.7629], 0.134, [273.7234, -0.2...","([0.001, 0.0529, 0.6739], 0.136, [312.1032, -0...","([0.0, 0.0243, 0.1813, 0.6015], 0.185, [331.92...","([0.0007, 0.0262, 0.2305, 0.9352], 0.177, [328...","([0.0012, 0.0901, 0.6126, 0.5601], 0.144, [324...","([0.0008, 0.0463, 0.2347, 0.6025, 0.9056], 0.1..."
4,사무통신(52),"([0.0, 0.0], 0.516, [9938.4334, -11.2115])","([0.0, 0.0, 0.5871], 0.521, [9505.7891, -10.90...","([0.0, 0.0, 0.3106], 0.532, [9743.016, -10.830...","([0.0, 0.0, 0.0739], 0.564, [7341.5665, -12.03...","([0.0, 0.0, 0.7014, 0.3549], 0.535, [9445.4651...","([0.0, 0.0, 0.8782, 0.0926], 0.565, [7376.5461...","([0.0, 0.0, 0.7265, 0.1364], 0.566, [7489.4149...","([0.0001, 0.0, 0.8776, 0.7299, 0.1548], 0.566,..."
5,요식업소(80),"([0.0, 0.0], 0.431, [3798.3912, -5.1244])","([0.0, 0.0001, 0.0327], 0.51, [2906.9741, -4.4...","([0.0, 0.0001, 0.3154], 0.449, [3693.3997, -4....","([0.007, 0.0, 0.0508], 0.498, [2266.3714, -5.6...","([0.0, 0.0002, 0.0471, 0.4766], 0.518, [2883.4...","([0.0117, 0.0001, 0.1258, 0.2025], 0.536, [208...","([0.0086, 0.0, 0.7852, 0.0949], 0.499, [2328.2...","([0.014, 0.0002, 0.1325, 0.791, 0.2835], 0.537..."


In [59]:
# 대구 카드 휴일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekend_seg2_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekend_19_20_seg2['confirmed'])
  X2 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekend_19_20_seg2[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X3)
  result4 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X6)
  result7 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekend_19_20_seg2[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 휴일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구휴일군집2-주거상권.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 휴일 군집2-주거상권


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,보건위생(71),"([0.0, 0.0095], 0.192, [618.1772, -0.5095])","([0.0, 0.0375, 0.0164], 0.331, [442.197, -0.38...","([0.0, 0.0157, 0.4811], 0.205, [605.0581, -0.4...","([0.002, 0.0072, 0.3883], 0.212, [495.177, -0....","([0.0, 0.0472, 0.0223, 0.7193], 0.334, [440.12...","([0.0031, 0.0554, 0.0276, 0.9678], 0.331, [446...","([0.0024, 0.0143, 0.6964, 0.5312], 0.216, [511...","([0.0036, 0.0849, 0.0302, 0.6919, 0.8542], 0.3..."
1,요식업소(80),"([0.0, 0.0001], 0.402, [1939.3157, -1.9798])","([0.0, 0.0003, 0.0502], 0.473, [1602.0442, -1....","([0.0, 0.0001, 0.3976], 0.416, [1902.9424, -1....","([0.0001, 0.0, 0.1247], 0.446, [1439.643, -2.1...","([0.0, 0.0004, 0.0677, 0.5718], 0.478, [1594.2...","([0.0002, 0.0002, 0.138, 0.3801], 0.486, [1364...","([0.0002, 0.0001, 0.7898, 0.1978], 0.448, [146...","([0.0004, 0.0006, 0.145, 0.7959, 0.4785], 0.48..."


In [60]:
# 대구 카드 휴일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekend_seg3_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekend_19_20_seg3['confirmed'])
  X2 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekend_19_20_seg3[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X3)
  result4 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X6)
  result7 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekend_19_20_seg3[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 휴일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구휴일군집3-주거.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 휴일 군집3-주거


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저업소(21),"([0.0, 0.0281], 0.142, [202.6675, -0.1676])","([0.0, 0.0796, 0.1032], 0.213, [154.321, -0.13...","([0.0, 0.0386, 0.7207], 0.146, [200.032, -0.16...","([0.0117, 0.0211, 0.3962], 0.162, [154.9216, -...","([0.0001, 0.0901, 0.1171, 0.9197], 0.214, [154...","([0.019, 0.0883, 0.1643, 0.8039], 0.215, [142....","([0.0164, 0.03, 0.9827, 0.4491], 0.162, [155.2...","([0.0258, 0.1066, 0.1717, 0.9924, 0.8234], 0.2..."
1,사무통신(52),"([0.6232, 0.0289], 0.141, [29.7837, 0.6227])","([0.5115, 0.05, 0.6278], 0.147, [84.592, 0.584...","([0.6485, 0.035, 0.9714], 0.141, [30.7715, 0.6...","([0.4355, 0.0546, 0.3411], 0.166, [-170.0994, ...","([0.5215, 0.0555, 0.633, 0.9656], 0.147, [84.1...","([0.5287, 0.1442, 0.3525, 0.2185], 0.19, [-139...","([0.3888, 0.09, 0.6518, 0.2989], 0.172, [-198....","([0.4736, 0.2026, 0.3612, 0.6586, 0.1981], 0.1..."
2,요식업소(80),"([0.0, 0.0001], 0.388, [1134.7627, -1.2335])","([0.0, 0.0004, 0.068], 0.451, [932.2848, -1.09...","([0.0, 0.0002, 0.2931], 0.41, [1105.8514, -1.1...","([0.0007, 0.0, 0.1016], 0.439, [793.771, -1.34...","([0.0, 0.0007, 0.0951, 0.4261], 0.463, [925.22...","([0.0012, 0.0003, 0.1921, 0.3003], 0.471, [751...","([0.0008, 0.0001, 0.6521, 0.1889], 0.443, [821...","([0.0015, 0.0007, 0.1996, 0.6567, 0.4303], 0.4..."


In [61]:
# 대구 카드 휴일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for i in card_daegu_weekend_seg4_corr40_list:
  i = i[-3:-1]
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X
  X1 = sm.add_constant(card_daegu_weekend_19_20_seg4['confirmed'])
  X2 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'temp_mean']])
  X3 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'rain_mean']])
  X4 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'hum_mean']])
  X5 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(card_daegu_weekend_19_20_seg4[['confirmed', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X1) 
  result2 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X2) 
  result3 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X3)
  result4 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X4) 
  result5 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X5) 
  result6 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X6)
  result7 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X7) 
  result8 = sm.OLS(card_daegu_weekend_19_20_seg4[x2020], X8)

  business_type.append(type_list[i])

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 카드 휴일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['card_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'card_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

card_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/카드대구휴일군집4-교통중심.csv',index=False, encoding='cp949')

pvalue_test

대구 카드 휴일 군집4-교통중심


,card_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,레저업소(21),"([0.0, 0.0373], 0.129, [343.4513, -0.3822])","([0.0, 0.025, 0.3364], 0.155, [413.2961, -0.43...","([0.0, 0.058, 0.4237], 0.147, [329.2687, -0.35...","([0.1973, 0.0219, 0.2315], 0.169, [181.7852, -...","([0.0, 0.0362, 0.2699, 0.3316], 0.181, [407.55...","([0.1242, 0.0069, 0.1144, 0.0841], 0.236, [215...","([0.1879, 0.038, 0.7224, 0.345], 0.172, [195.9...","([0.1195, 0.0128, 0.1191, 0.705, 0.1458], 0.24..."
1,연료판매(33),"([0.0, 0.0041], 0.23, [2129.8075, -0.826])","([0.0, 0.002, 0.198], 0.271, [2270.83, -0.9253...","([0.0, 0.004, 0.581], 0.238, [2144.6893, -0.85...","([0.0, 0.0102, 0.1949], 0.272, [2394.933, -0.7...","([0.0, 0.0022, 0.2315, 0.7249], 0.274, [2273.9...","([0.0, 0.0078, 0.3938, 0.3867], 0.289, [2422.5...","([0.0, 0.0148, 0.9654, 0.2484], 0.272, [2392.3...","([0.0, 0.0112, 0.4021, 0.9716, 0.4365], 0.289,..."
2,의복(42),"([0.0, 0.0], 0.422, [1369.5002, 4.2062])","([0.0001, 0.0001, 0.2973], 0.442, [1743.9401, ...","([0.0, 0.0001, 0.8722], 0.422, [1383.6917, 4.1...","([0.0004, 0.0, 0.062], 0.484, [2597.4136, 4.59...","([0.0002, 0.0002, 0.3112, 0.9937], 0.442, [174...","([0.0005, 0.0, 0.7115, 0.1172], 0.486, [2635.5...","([0.0005, 0.0, 0.5199, 0.0526], 0.491, [2719.9...","([0.0005, 0.0001, 0.71, 0.5239, 0.0956], 0.494..."
3,요식업소(80),"([0.0, 0.0], 0.438, [1477.6045, -1.7943])","([0.0, 0.0001, 0.1201], 0.481, [1249.6775, -1....","([0.0, 0.0, 0.3707], 0.452, [1445.2246, -1.731...","([0.0009, 0.0, 0.0903], 0.488, [1015.4896, -1....","([0.0, 0.0002, 0.1567, 0.5048], 0.488, [1241.7...","([0.0015, 0.0001, 0.3208, 0.2328], 0.505, [973...","([0.0012, 0.0, 0.7977, 0.1507], 0.489, [1035.9...","([0.0021, 0.0002, 0.3297, 0.8046, 0.3157], 0.5..."


## 편의점 데이터 전처리
- 서울 평일, 휴일
- 대구 평일, 휴일

In [62]:
# gs 전처리
gs_row = gs_row.rename(columns={'OPER_DT':'STD_DD', 'ADMD':'HDONG_NM'})
gs_row2020_ls = gs_row['STD_DD']>20191231
gs2020 = gs_row[gs_row2020_ls]
gs2020.reset_index(inplace=True, drop=True)
del gs2020['PVN']; del gs2020['BOR']; #del gs2020['AMT_IND']

# gs2020
gs2020 = pd.merge(holiday, gs2020, on='STD_DD')
gs2020 = pd.merge(covid, gs2020, on='STD_DD' )

# 카테고리 비중이 문자열로 된 것을 변환
index_list=['LCLS_10_P', 'LCLS_20_P', 'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P','LCLS_80_P']

# 각 동의 전체 매출에 대한 카테고리별 퍼센트로 나와있어서 이것을 전체 매출과 곱하여 카테고리별 절대 지수를 만듬  
for i in index_list:
  gs2020['mul_'+i]=gs2020['AMT_IND']*gs2020[i]*100
  del gs2020[i]

# 평일 휴일 나누기
weekend = gs2020[gs2020['weekend']==1].index
gs2020_weekday = gs2020.drop(weekend)

# 휴일 데이터
weekday = gs2020[gs2020['weekend'] == 0].index
gs2020_weekend = gs2020.drop(weekday)

gs2020_weekday.reset_index(inplace=True, drop=True)
gs2020_weekend.reset_index(inplace=True, drop=True)

print(gs2020.shape, gs2020_weekday.shape, gs2020_weekend.shape)

(7260, 14) (4920, 14) (2340, 14)


In [63]:
# gs 2019 처리
gs_row2019_ls = gs_row['STD_DD']<20191231
gs2019 = gs_row[gs_row2019_ls]
gs2019.reset_index(inplace=True, drop=True)
del gs2019['PVN']; del gs2019['BOR']; #del gs2020['AMT_IND']

# gs2020
gs2019 = pd.merge(holiday, gs2019, on='STD_DD')

# 카테고리 비중이 문자열로 된 것을 변환
index_list=['LCLS_10_P', 'LCLS_20_P', 'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P','LCLS_80_P']

# 각 동의 전체 매출에 대한 카테고리별 퍼센트로 나와있어서 이것을 전체 매출과 곱하여 카테고리별 절대 지수를 만듬  
for i in index_list:
  gs2019['mul_'+i]=gs2019['AMT_IND']*gs2019[i]*100
  del gs2019[i]

# gs2019 데이터 2020으로 날짜 맞춰주기 timedelta 이용
gs2019['STD_DD'] = gs2019['STD_DD'].astype(str)
gs2019['STD_DD'] = pd.to_datetime(gs2019['STD_DD'])
gs2019['STD_DD'] = pd.DatetimeIndex(gs2019['STD_DD']) + timedelta(weeks=52)

gs2019['STD_DD'] = gs2019['STD_DD'].astype(str)
gs2019["STD_DD"] = gs2019["STD_DD"].str.replace("-","")
gs2019["STD_DD"] = gs2019["STD_DD"].astype(int)

# gs 19 20 날짜 합친 것.
gs_19_20 = pd.merge(gs2019, gs2020, on=['STD_DD', 'HDONG_NM'])
gs_19_20 = pd.merge(gs_19_20, covid, on = ['STD_DD'])

# 평일 휴일 나누기
gs_19_20_weekday = gs_19_20[gs_19_20['weekend_x']==0]
gs_19_20_weekend = gs_19_20[gs_19_20['weekend_x']==1]

gs_19_20_weekday.reset_index(inplace=True, drop=True)
gs_19_20_weekend.reset_index(inplace=True, drop=True)

print(gs_19_20.shape, gs_19_20_weekday.shape, gs_19_20_weekend.shape)
gs_19_20

(7140, 26) (4800, 26) (2340, 26)


,STD_DD,day_x,weekend_x,HDONG_NM,AMT_IND_x,mul_LCLS_10_P_x,mul_LCLS_20_P_x,mul_LCLS_30_P_x,mul_LCLS_40_P_x,mul_LCLS_50_P_x,mul_LCLS_60_P_x,mul_LCLS_70_P_x,mul_LCLS_80_P_x,confirmed_x,day_y,weekend_y,AMT_IND_y,mul_LCLS_10_P_y,mul_LCLS_20_P_y,mul_LCLS_30_P_y,mul_LCLS_40_P_y,mul_LCLS_50_P_y,mul_LCLS_60_P_y,mul_LCLS_70_P_y,mul_LCLS_80_P_y,confirmed_y
0,20200201,5,1,고산1동,0.940026,27.677739,18.980567,36.314502,2.729587,6.631926,0.609912,0.669998,0.388402,4,5,1,0.854239,22.660598,19.670760,31.086279,3.054371,7.555833,0.494051,0.569419,0.332563,4
1,20200201,5,1,고산2동,1.172479,44.389652,23.625214,35.771146,1.435410,7.662538,1.153544,1.887421,1.322987,4,5,1,1.080543,35.531166,27.467702,36.191390,1.101706,5.976038,0.000000,1.178058,0.608279,4
2,20200201,5,1,고산3동,0.895912,34.374076,18.128174,28.175804,0.686017,4.996726,0.881955,1.806460,0.541952,4,5,1,0.759088,23.650104,16.925769,28.587669,0.990256,4.197883,0.000000,1.217369,0.339751,4
3,20200201,5,1,두산동,1.092902,24.283429,24.905694,45.970941,2.457469,9.456863,0.682309,0.593916,0.939606,4,5,1,0.911892,17.675688,20.850350,31.331062,3.068180,14.231147,0.090823,2.876597,1.065312,4
4,20200201,5,1,만촌1동,1.253741,50.045001,25.991590,40.460214,3.107759,4.421891,0.898299,0.206108,0.243204,4,5,1,1.124104,34.438142,28.609536,35.419524,2.716673,7.735932,1.173468,0.539972,1.777157,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7135,20200529,4,0,중림동,1.104929,33.587966,24.480168,44.267867,2.612954,4.576105,0.000000,0.866170,0.101623,79,4,0,1.156155,37.579270,25.444363,40.216927,2.546906,6.993686,2.032528,0.801787,0.000000,79
7136,20200529,4,0,청구동,1.464850,43.946314,31.438228,57.398650,6.060868,4.589281,1.883208,0.500717,0.667784,79,4,0,1.563844,43.949677,39.068269,58.904006,6.319895,7.282288,0.088826,0.029429,0.741977,79
7137,20200529,4,0,필동,1.734506,53.086083,34.561248,66.755000,5.836981,9.891391,0.948948,1.518038,0.852941,79,4,0,1.285247,39.146678,26.147883,47.763488,5.092193,8.392937,0.697045,1.240833,0.043642,79
7138,20200529,4,0,황학동,1.163772,33.133533,24.697378,45.401658,3.784102,6.149560,1.245609,0.848000,1.117311,79,4,0,1.155756,33.161728,26.420707,45.185221,3.688446,6.616800,0.149707,0.000000,0.353007,79


### 편의점 서울 평일, 휴일

In [64]:
# 각 cluster와 서울 편의점 평일 교집합 
seoul_weekday_seg1_list = list(set(seoul_cluster1).intersection(set(gs_weekday_list)))
seoul_weekday_seg2_list = list(set(seoul_cluster2).intersection(set(gs_weekday_list)))
seoul_weekday_seg3_list = list(set(seoul_cluster3).intersection(set(gs_weekday_list)))
seoul_weekday_seg4_list = list(set(seoul_cluster4).intersection(set(gs_weekday_list)))
print(seoul_weekday_seg1_list, '\n', seoul_weekday_seg2_list, '\n', seoul_weekday_seg3_list, '\n', seoul_weekday_seg4_list)

# 서울 평일 각 군집 로 분리 데이터프레임 생성
gs_seoul_weekday_seg1 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg1_list)]
gs_seoul_weekday_seg2 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg2_list)] 
gs_seoul_weekday_seg3 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg3_list)]
gs_seoul_weekday_seg4 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg4_list)]

print(gs_seoul_weekday_seg1.shape, gs_seoul_weekday_seg2.shape, gs_seoul_weekday_seg3.shape, gs_seoul_weekday_seg4.shape)

# 평일 각 군집 별 일별 업종별 
gs_seoul_weekday_seg1_corr = gs_seoul_weekday_seg1.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekday_seg2_corr = gs_seoul_weekday_seg2.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekday_seg3_corr = gs_seoul_weekday_seg3.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekday_seg4_corr = gs_seoul_weekday_seg4.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_seoul_weekday_seg1_corr.shape, gs_seoul_weekday_seg2_corr.shape, gs_seoul_weekday_seg3_corr.shape, gs_seoul_weekday_seg4_corr.shape)

['필동', '상계2동', '회현동', '소공동'] 
 ['하계1동', '중림동', '상계1동', '월계1동', '상계9동', '중계1동'] 
 ['상계10동', '청구동', '월계3동', '상계3.4동', '상계5동'] 
 ['신당동', '장충동', '공릉2동', '신당5동', '을지로동', '약수동', '황학동', '공릉1동', '상계6.7동']
(320, 26) (480, 26) (400, 26) (720, 26)
(80, 25) (80, 25) (80, 25) (80, 25)


In [65]:
# 각 cluster와 서울 편의점 휴일 교집합 
seoul_weekend_seg1_list = list(set(seoul_cluster1).intersection(set(gs_weekend_list)))
seoul_weekend_seg2_list = list(set(seoul_cluster2).intersection(set(gs_weekend_list)))
seoul_weekend_seg3_list = list(set(seoul_cluster3).intersection(set(gs_weekend_list)))
seoul_weekend_seg4_list = list(set(seoul_cluster4).intersection(set(gs_weekend_list)))
print(seoul_weekend_seg1_list, '\n', seoul_weekend_seg2_list, '\n', seoul_weekend_seg3_list, '\n', seoul_weekend_seg4_list)

# 서울 휴일 각 군집 로 분리 데이터프레임 생성
gs_seoul_weekend_seg1 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg1_list)]
gs_seoul_weekend_seg2 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg2_list)] 
gs_seoul_weekend_seg3 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg3_list)]
gs_seoul_weekend_seg4 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg4_list)]

print(gs_seoul_weekend_seg1.shape, gs_seoul_weekend_seg2.shape, gs_seoul_weekend_seg3.shape, gs_seoul_weekend_seg4.shape)

# 휴일 각 군집 별 일별 업종별 
gs_seoul_weekend_seg1_corr = gs_seoul_weekend_seg1.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekend_seg2_corr = gs_seoul_weekend_seg2.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekend_seg3_corr = gs_seoul_weekend_seg3.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekend_seg4_corr = gs_seoul_weekend_seg4.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_seoul_weekend_seg1_corr.shape, gs_seoul_weekend_seg2_corr.shape, gs_seoul_weekend_seg3_corr.shape, gs_seoul_weekend_seg4_corr.shape)

['필동', '명동', '상계2동', '회현동', '소공동'] 
 ['하계1동', '중계본동', '중림동', '상계1동', '월계1동', '중계4동', '상계9동', '중계1동'] 
 ['상계10동', '청구동', '월계3동', '하계2동', '월계2동', '상계5동'] 
 ['신당동', '장충동', '공릉2동', '신당5동', '광희동', '을지로동', '약수동', '황학동', '공릉1동', '상계6.7동']
(195, 26) (312, 26) (234, 26) (390, 26)
(39, 25) (39, 25) (39, 25) (39, 25)


### 편의점 대구 평일, 휴일

In [66]:
# 각 cluster와 대구 편의점 평일 교집합 
daegu_weekday_seg1_list = list(set(daegu_cluster1).intersection(set(gs_weekday_list)))
daegu_weekday_seg2_list = list(set(daegu_cluster2).intersection(set(gs_weekday_list)))
daegu_weekday_seg3_list = list(set(daegu_cluster3).intersection(set(gs_weekday_list)))
daegu_weekday_seg4_list = list(set(daegu_cluster4).intersection(set(gs_weekday_list)))
print(daegu_weekday_seg1_list, '\n', daegu_weekday_seg2_list, '\n', daegu_weekday_seg3_list, '\n', daegu_weekday_seg4_list)

# 대구 평일 각 군집 로 분리 데이터프레임 생성
gs_daegu_weekday_seg1 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg1_list)]
gs_daegu_weekday_seg2 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg2_list)] 
gs_daegu_weekday_seg3 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg3_list)]
gs_daegu_weekday_seg4 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg4_list)]

print(gs_daegu_weekday_seg1.shape, gs_daegu_weekday_seg2.shape, gs_daegu_weekday_seg3.shape, gs_daegu_weekday_seg4.shape)

# 휴일 각 군집 별 일별 업종별 
gs_daegu_weekday_seg1_corr = gs_daegu_weekday_seg1.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekday_seg2_corr = gs_daegu_weekday_seg2.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekday_seg3_corr = gs_daegu_weekday_seg3.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekday_seg4_corr = gs_daegu_weekday_seg4.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_daegu_weekday_seg1_corr.shape, gs_daegu_weekday_seg2_corr.shape, gs_daegu_weekday_seg3_corr.shape, gs_daegu_weekday_seg4_corr.shape)

['중동', '성내1동', '범어1동', '삼덕동', '두산동', '범어2동'] 
 ['남산4동', '고산3동', '범물1동', '범어3동', '범어4동', '만촌3동', '고산1동'] 
 ['수성1가동', '수성4가동', '황금1동'] 
 ['황금2동', '성내3동', '남산1동', '동인동', '고산2동']
(480, 26) (560, 26) (240, 26) (400, 26)
(80, 25) (80, 25) (80, 25) (80, 25)


In [67]:
# 각 cluster와 대구 편의점 휴일 교집합 
daegu_weekend_seg1_list = list(set(daegu_cluster1).intersection(set(gs_weekend_list)))
daegu_weekend_seg2_list = list(set(daegu_cluster2).intersection(set(gs_weekend_list)))
daegu_weekend_seg3_list = list(set(daegu_cluster3).intersection(set(gs_weekend_list)))
daegu_weekend_seg4_list = list(set(daegu_cluster4).intersection(set(gs_weekend_list)))
print(daegu_weekend_seg1_list, '\n', daegu_weekend_seg2_list, '\n', daegu_weekend_seg3_list, '\n', daegu_weekend_seg4_list)

# 대구 휴일 각 군집 로 분리 데이터프레임 생성
gs_daegu_weekend_seg1 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg1_list)]
gs_daegu_weekend_seg2 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg2_list)] 
gs_daegu_weekend_seg3 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg3_list)]
gs_daegu_weekend_seg4 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg4_list)]

print(gs_daegu_weekend_seg1.shape, gs_daegu_weekend_seg2.shape, gs_daegu_weekend_seg3.shape, gs_daegu_weekend_seg4.shape)

# 휴일 각 군집 별 일별 업종별 
gs_daegu_weekend_seg1_corr = gs_daegu_weekend_seg1.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekend_seg2_corr = gs_daegu_weekend_seg2.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekend_seg3_corr = gs_daegu_weekend_seg3.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekend_seg4_corr = gs_daegu_weekend_seg4.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_daegu_weekend_seg1_corr.shape, gs_daegu_weekend_seg2_corr.shape, gs_daegu_weekend_seg3_corr.shape, gs_daegu_weekend_seg4_corr.shape)

['중동', '성내1동', '범어1동', '삼덕동', '두산동', '성내2동', '범어2동'] 
 ['남산4동', '고산3동', '범물1동', '범어3동', '범어4동', '만촌3동', '고산1동'] 
 ['수성1가동', '수성4가동', '황금1동'] 
 ['수성2.3가동', '만촌1동', '성내3동', '남산1동', '동인동', '고산2동']
(273, 26) (273, 26) (117, 26) (234, 26)
(39, 25) (39, 25) (39, 25) (39, 25)


### 편의점 날씨 데이터 합침

In [68]:
# gs 서울 / gs 대구 / pvalue_test list
gs_seoul_pvalue_test = []
gs_daegu_pvalue_test = []

# 서울 카드 평일 + 서울 날씨 데이터
gs_seoul_weekday_seg1_corr = pd.merge(gs_seoul_weekday_seg1_corr, seoul_weather, on='STD_DD')
gs_seoul_weekday_seg2_corr = pd.merge(gs_seoul_weekday_seg2_corr, seoul_weather, on='STD_DD')
gs_seoul_weekday_seg3_corr = pd.merge(gs_seoul_weekday_seg3_corr, seoul_weather, on='STD_DD')
gs_seoul_weekday_seg4_corr = pd.merge(gs_seoul_weekday_seg4_corr, seoul_weather, on='STD_DD')

# 서울 카드 휴일
gs_seoul_weekend_seg1_corr = pd.merge(gs_seoul_weekend_seg1_corr, seoul_weather, on='STD_DD')
gs_seoul_weekend_seg2_corr = pd.merge(gs_seoul_weekend_seg2_corr, seoul_weather, on='STD_DD')
gs_seoul_weekend_seg3_corr = pd.merge(gs_seoul_weekend_seg3_corr, seoul_weather, on='STD_DD')
gs_seoul_weekend_seg4_corr = pd.merge(gs_seoul_weekend_seg4_corr, seoul_weather, on='STD_DD')

# 대구 카드 평일
gs_daegu_weekday_seg1_corr = pd.merge(gs_daegu_weekday_seg1_corr, daegu_weather, on='STD_DD')
gs_daegu_weekday_seg2_corr = pd.merge(gs_daegu_weekday_seg2_corr, daegu_weather, on='STD_DD')
gs_daegu_weekday_seg3_corr = pd.merge(gs_daegu_weekday_seg3_corr, daegu_weather, on='STD_DD')
gs_daegu_weekday_seg4_corr = pd.merge(gs_daegu_weekday_seg4_corr, daegu_weather, on='STD_DD')

# 대구 카드 휴일
gs_daegu_weekend_seg1_corr = pd.merge(gs_daegu_weekend_seg1_corr, daegu_weather, on='STD_DD')
gs_daegu_weekend_seg2_corr = pd.merge(gs_daegu_weekend_seg2_corr, daegu_weather, on='STD_DD')
gs_daegu_weekend_seg3_corr = pd.merge(gs_daegu_weekend_seg3_corr, daegu_weather, on='STD_DD')
gs_daegu_weekend_seg4_corr = pd.merge(gs_daegu_weekend_seg4_corr, daegu_weather, on='STD_DD')

# example
gs_seoul_weekday_seg1_corr

,STD_DD,day_x,weekend_x,AMT_IND_x,mul_LCLS_10_P_x,mul_LCLS_20_P_x,mul_LCLS_30_P_x,mul_LCLS_40_P_x,mul_LCLS_50_P_x,mul_LCLS_60_P_x,mul_LCLS_70_P_x,mul_LCLS_80_P_x,confirmed_x,day_y,weekend_y,AMT_IND_y,mul_LCLS_10_P_y,mul_LCLS_20_P_y,mul_LCLS_30_P_y,mul_LCLS_40_P_y,mul_LCLS_50_P_y,mul_LCLS_60_P_y,mul_LCLS_70_P_y,mul_LCLS_80_P_y,confirmed_y,temp_mean,rain_mean,hum_mean
0,20200206,3,0,1.387006,42.325254,26.081096,51.057994,4.832977,11.405558,0.760213,1.846822,0.390673,5,3,0,1.595112,47.820845,33.756347,51.681922,4.962981,18.816715,0.521515,1.611634,0.339199,5,-6.4,0.0,36.4
1,20200207,4,0,1.522121,45.659766,30.187557,56.675373,4.564837,10.068387,0.575422,4.147122,0.333589,3,4,0,1.497372,45.262571,32.438955,49.380296,4.642120,15.399561,0.672748,1.524275,0.416668,3,-1.7,0.0,45.1
2,20200210,0,0,1.565045,47.737979,33.509245,55.167054,6.709560,10.290072,0.660270,1.972588,0.457687,3,0,0,1.511974,45.008138,33.456115,50.799869,7.657671,11.333158,0.362103,2.200163,0.380199,3,4.0,0.0,49.4
3,20200211,1,0,1.659646,49.622971,38.937166,59.557797,5.738430,9.080958,0.758130,1.920124,0.349062,0,1,0,1.533940,44.783325,34.389101,55.039395,5.522098,11.376661,0.562330,1.307966,0.413150,0,6.8,0.0,37.0
4,20200212,2,0,1.829900,49.503211,57.279342,58.801301,4.665383,9.154791,0.774801,2.405707,0.405437,1,2,0,1.606864,46.137439,33.839846,51.050927,4.895450,22.723790,0.760011,0.957566,0.321380,1,7.4,6.7,73.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,20200525,0,0,1.624822,51.117086,29.313333,57.776352,5.987576,15.586530,0.589132,1.720114,0.392072,25,0,0,1.301989,40.196050,24.588662,49.448730,5.643477,8.283627,0.637524,1.314935,0.085851,25,16.6,0.0,81.0
76,20200526,1,0,1.710506,49.447702,33.003597,69.888515,6.656267,8.921707,0.587098,1.971054,0.574619,16,1,0,1.338201,38.240122,24.429909,49.699198,4.522319,15.144718,0.480383,0.982377,0.321074,16,16.2,1.5,81.9
77,20200527,2,0,1.735761,49.924435,34.772690,71.394608,4.697267,9.949176,0.550655,1.770385,0.516932,19,2,0,1.323768,38.680964,25.790472,53.170236,3.546766,9.348854,0.480119,0.970491,0.388851,19,17.4,0.0,68.5
78,20200528,3,0,1.748042,51.533489,33.627375,69.759000,6.142264,10.418030,0.721515,2.192932,0.409628,40,3,0,1.353160,39.393586,26.616203,53.754560,4.655036,9.006488,0.486010,1.277665,0.126465,40,18.7,0.0,62.8


## 편의점 회귀모델

### 서울 편의점 평일 군집1-4

In [69]:
# 서울 편의점 평일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekday_seg1_corr40_list:
  i = circulation_list_reverse[j]
  
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekday_seg1_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekday_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekday_seg1_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 평일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울평일군집1-상권.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 평일 군집1-상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,마실거리,"([0.0, 0.0], 0.198, [47.0801, -0.0179])","([0.0, 0.0002, 0.4587], 0.203, [46.2506, -0.01...","([0.0, 0.0, 0.3978], 0.205, [47.2386, -0.018, ...","([0.0, 0.0001, 0.5946], 0.201, [45.954, -0.017...","([0.0, 0.0003, 0.3897, 0.3418], 0.213, [46.287...","([0.0, 0.0003, 0.4998, 0.6605], 0.205, [45.374...","([0.0, 0.0001, 0.218, 0.2961], 0.217, [44.8466...","([0.0, 0.0003, 0.4333, 0.1973, 0.3269], 0.223,..."


In [70]:
# 서울 편의점 평일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekday_seg2_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekday_seg2_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekday_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekday_seg2_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 평일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울평일군집2-주거상권.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 평일 군집2-주거상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,식사,"([0.0, 0.0003], 0.156, [28.8946, -0.0044])","([0.0, 0.0018, 0.3214], 0.166, [28.5774, -0.00...","([0.0, 0.0003, 0.114], 0.183, [28.8103, -0.004...","([0.0, 0.0005, 0.0228], 0.211, [27.5368, -0.00...","([0.0, 0.0016, 0.4262, 0.1447], 0.19, [28.5613...","([0.0, 0.002, 0.467, 0.0305], 0.216, [27.3636,...","([0.0, 0.0005, 0.5404, 0.0818], 0.215, [27.69,...","([0.0, 0.0019, 0.4994, 0.5814, 0.0931], 0.22, ..."


In [71]:
# 서울 편의점 평일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekday_seg3_corr40_list:
  i = circulation_list_reverse[j]
  
  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekday_seg3_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekday_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekday_seg3_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 평일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울평일군집3-주거.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 평일 군집3-주거


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh


In [72]:
# 서울 편의점 평일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekday_seg4_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekday_seg4_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekday_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekday_seg4_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 평일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울평일군집4-교통중심.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 평일 군집4-교통중심


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh


### 서울 편의점 휴일 군집1-4

In [73]:
# 서울 편의점 휴일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekend_seg1_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekend_seg1_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekend_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekend_seg1_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 휴일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울휴일군집1-상권.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 휴일 군집1-상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,취미&여가활동_비중,"([0.0, 0.02], 0.138, [0.4208, -0.0002])","([0.0, 0.0162, 0.3646], 0.157, [0.4402, -0.000...","([0.0, 0.0312, 0.1244], 0.193, [0.4089, -0.000...","([0.0, 0.0228, 0.7124], 0.141, [0.4013, -0.000...","([0.0, 0.0242, 0.291, 0.1061], 0.219, [0.4305,...","([0.0, 0.0165, 0.239, 0.3963], 0.175, [0.3994,...","([0.0, 0.0341, 0.1168, 0.5863], 0.2, [0.4397, ...","([0.0, 0.0272, 0.3716, 0.1749, 0.9519], 0.219,..."
1,임신/육아_비중,"([0.0, 0.0027], 0.219, [0.3776, -0.0004])","([0.0, 0.0026, 0.5728], 0.226, [0.3963, -0.000...","([0.0, 0.0034, 0.951], 0.219, [0.3769, -0.0004...","([0.0002, 0.0032, 0.7265], 0.221, [0.3491, -0....","([0.0, 0.0033, 0.5736, 0.9152], 0.226, [0.3953...","([0.0003, 0.0028, 0.4306, 0.5114], 0.235, [0.3...","([0.0009, 0.0037, 0.8811, 0.7107], 0.222, [0.3...","([0.0014, 0.0031, 0.4094, 0.7531, 0.4766], 0.2..."


In [74]:
# 서울 편의점 휴일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekend_seg2_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekend_seg2_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekend_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekend_seg2_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 휴일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울휴일군집2-주거상권.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 휴일 군집2-주거상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,식사,"([0.0, 0.0236], 0.131, [29.5355, -0.0033])","([0.0, 0.0304, 0.6202], 0.137, [29.3553, -0.00...","([0.0, 0.022, 0.5818], 0.138, [29.6083, -0.003...","([0.0, 0.0263, 0.1672], 0.176, [28.3186, -0.00...","([0.0, 0.0282, 0.5914, 0.5573], 0.146, [29.416...","([0.0, 0.0287, 0.9106, 0.202], 0.177, [28.3155...","([0.0, 0.0151, 0.1151, 0.0444], 0.234, [27.683...","([0.0, 0.0149, 0.6549, 0.107, 0.0499], 0.238, ..."
1,임신/육아_비중,"([0.0, 0.0088], 0.172, [0.5359, -0.0004])","([0.0, 0.0132, 0.0215], 0.286, [0.45, -0.0004,...","([0.0, 0.0049, 0.132], 0.223, [0.5568, -0.0004...","([0.0, 0.01, 0.9102], 0.172, [0.5251, -0.0004,...","([0.0, 0.0063, 0.0126, 0.0697], 0.351, [0.4683...","([0.0, 0.0134, 0.0131, 0.3102], 0.307, [0.5323...","([0.0002, 0.0044, 0.0638, 0.2647], 0.25, [0.44...","([0.0, 0.0073, 0.0279, 0.1384, 0.99], 0.351, [..."


In [75]:
# 서울 편의점 휴일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekend_seg3_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekend_seg3_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekend_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekend_seg3_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 휴일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울휴일군집3-주거.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 휴일 군집3-주거


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh


In [76]:
# 서울 편의점 휴일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_seoul_weekend_seg4_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_seoul_weekend_seg4_corr['confirmed_y'])
  X2 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_seoul_weekend_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X1) 
  result2 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X2) 
  result3 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X3)
  result4 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X4) 
  result5 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X5) 
  result6 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X6)
  result7 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X7) 
  result8 = sm.OLS(gs_seoul_weekend_seg4_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('서울 편의점 휴일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_seoul_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점서울휴일군집4-교통중심.csv',index=False, encoding='cp949')
pvalue_test

서울 편의점 휴일 군집4-교통중심


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,마실거리,"([0.0, 0.0089], 0.171, [33.2875, -0.008])","([0.0, 0.0136, 0.0274], 0.277, [31.691, -0.007...","([0.0, 0.0045, 0.0993], 0.232, [33.7281, -0.00...","([0.0, 0.0099, 0.1467], 0.219, [30.6635, -0.00...","([0.0, 0.0058, 0.0152, 0.0513], 0.352, [32.072...","([0.0, 0.0144, 0.0818, 0.5566], 0.284, [30.759...","([0.0, 0.0013, 0.002, 0.0028], 0.407, [28.2363...","([0.0, 0.0023, 0.192, 0.0047, 0.031], 0.436, [..."
1,임신/육아_비중,"([0.0, 0.0017], 0.235, [0.6692, -0.0006])","([0.0, 0.0014, 0.0003], 0.474, [0.5258, -0.000...","([0.0, 0.0006, 0.0483], 0.315, [0.6994, -0.000...","([0.0, 0.0019, 0.756], 0.237, [0.7021, -0.0006...","([0.0, 0.0002, 0.0, 0.0058], 0.578, [0.5527, -...","([0.0, 0.0007, 0.0, 0.0153], 0.556, [0.7148, -...","([0.0, 0.0006, 0.0335, 0.3647], 0.331, [0.5995...","([0.0, 0.0003, 0.0, 0.0833, 0.2505], 0.594, [0..."


### 대구 편의점 평일 군집1-4

In [77]:
# 대구 편의점 평일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekday_seg1_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekday_seg1_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekday_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekday_seg1_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 편의점 평일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구평일군집1-상권.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 평일 군집1-상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,식사,"([0.0, 0.0001], 0.191, [26.1246, -0.0102])","([0.0, 0.0, 0.0006], 0.305, [28.8181, -0.0131,...","([0.0, 0.0001, 0.9972], 0.191, [26.1243, -0.01...","([0.0, 0.0001, 0.4877], 0.196, [26.9555, -0.01...","([0.0, 0.0, 0.0006, 0.6839], 0.307, [28.8117, ...","([0.0, 0.0, 0.0008, 0.699], 0.307, [29.2224, -...","([0.0, 0.0001, 0.6827, 0.4229], 0.198, [27.191...","([0.0, 0.0, 0.0007, 0.4902, 0.4976], 0.311, [2..."
1,마실거리,"([0.0, 0.0], 0.377, [33.7989, -0.0303])","([0.0, 0.0, 0.0], 0.554, [26.7511, -0.0227, 0....","([0.0, 0.0, 0.4504], 0.382, [33.9514, -0.0304,...","([0.0, 0.0, 0.1452], 0.394, [37.0007, -0.0295,...","([0.0, 0.0, 0.0, 0.1263], 0.568, [26.7914, -0....","([0.0, 0.0, 0.0, 0.021], 0.585, [30.7487, -0.0...","([0.0, 0.0, 0.9793, 0.2158], 0.394, [36.9733, ...","([0.0, 0.0, 0.0, 0.6482, 0.0756], 0.586, [30.3..."
2,홈&리빙,"([0.0, 0.0009], 0.133, [3.2462, -0.0021])","([0.0, 0.0138, 0.0186], 0.193, [2.7724, -0.001...","([0.0, 0.001, 0.8073], 0.133, [3.2395, -0.0021...","([0.0, 0.001, 0.9246], 0.133, [3.2178, -0.0021...","([0.0, 0.0144, 0.0201, 0.9848], 0.193, [2.7724...","([0.0, 0.0163, 0.0193, 0.8847], 0.193, [2.812,...","([0.0, 0.0012, 0.8214, 0.9737], 0.133, [3.2505...","([0.0, 0.0175, 0.0207, 0.9512, 0.8765], 0.193,..."


In [78]:
# 대구 편의점 평일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekday_seg2_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekday_seg2_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekday_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekday_seg2_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 편의점 평일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구평일군집2-주거상권.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 평일 군집2-주거상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,마실거리,"([0.0, 0.0], 0.351, [27.5744, -0.019])","([0.0, 0.0, 0.0], 0.563, [22.5739, -0.0136, 0....","([0.0, 0.0, 0.2156], 0.364, [27.7396, -0.0191,...","([0.0, 0.0, 0.053], 0.382, [30.3777, -0.0183, ...","([0.0, 0.0, 0.0, 0.0259], 0.591, [22.6112, -0....","([0.0, 0.0, 0.0, 0.0024], 0.613, [25.9102, -0....","([0.0, 0.0, 0.7535, 0.1311], 0.383, [30.1607, ...","([0.0, 0.0, 0.0, 0.3781, 0.026], 0.617, [25.38..."


In [79]:
# 대구 편의점 평일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekday_seg3_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekday_seg3_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekday_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])
  try:
    result1 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X1) 
    result2 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X2) 
    result3 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X3)
    result4 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X4) 
    result5 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X5) 
    result6 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X6)
    result7 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X7) 
    result8 = sm.OLS(gs_daegu_weekday_seg3_corr[x2020], X8)


    business_type.append(j)

    model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
    model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
    model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
    model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
    model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
    model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
    model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
    model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))
  except:
    pass
# 최종 Data Frame
print('대구 편의점 평일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구평일군집3-주거.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 평일 군집3-주거


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh


In [80]:
# 대구 편의점 평일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekday_seg4_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekday_seg4_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekday_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekday_seg4_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 편의점 평일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구평일군집4-교통중심.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 평일 군집4-교통중심


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,마실거리,"([0.0, 0.0], 0.303, [32.4713, -0.0192])","([0.0, 0.0, 0.0], 0.625, [25.7569, -0.012, 0.5...","([0.0, 0.0, 0.2171], 0.317, [32.6571, -0.0193,...","([0.0, 0.0, 0.0307], 0.344, [35.9936, -0.0183,...","([0.0, 0.0, 0.0, 0.0079], 0.659, [25.8015, -0....","([0.0, 0.0, 0.0, 0.0001], 0.694, [30.0092, -0....","([0.0, 0.0, 0.8565, 0.0765], 0.344, [35.8535, ...","([0.0, 0.0, 0.0, 0.3491, 0.0027], 0.698, [29.4..."


### 대구 편의점 휴일 군집1-4

In [81]:
# 대구 편의점 휴일 군집1
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekend_seg1_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekend_seg1_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekend_seg1_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekend_seg1_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 편의점 휴일 군집1-상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구휴일군집1-상권.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 휴일 군집1-상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,간식,"([0.0, 0.0091], 0.17, [18.4914, -0.0107])","([0.0, 0.0069, 0.3963], 0.187, [19.6441, -0.01...","([0.0, 0.009, 0.6351], 0.175, [18.6646, -0.010...","([0.0, 0.0382, 0.1268], 0.223, [22.721, -0.008...","([0.0, 0.0073, 0.4392, 0.7312], 0.189, [19.696...","([0.0, 0.0545, 0.9117, 0.209], 0.223, [22.716,...","([0.0, 0.053, 0.8853, 0.1505], 0.223, [22.8394...","([0.0, 0.0728, 0.9188, 0.8914, 0.2307], 0.223,..."
1,마실거리,"([0.0, 0.0], 0.371, [31.358, -0.0245])","([0.0, 0.0001, 0.0168], 0.464, [27.1238, -0.02...","([0.0, 0.0, 0.7562], 0.373, [31.511, -0.0247, ...","([0.0, 0.0, 0.3993], 0.383, [28.1712, -0.026, ...","([0.0, 0.0001, 0.0143, 0.464], 0.473, [27.263,...","([0.0, 0.0009, 0.0252, 0.7039], 0.467, [28.304...","([0.0, 0.0, 0.4799, 0.2952], 0.392, [27.3733, ...","([0.0, 0.001, 0.0289, 0.5308, 0.9108], 0.473, ..."
2,홈&리빙,"([0.0, 0.0023], 0.224, [2.9581, -0.002])","([0.0, 0.0027, 0.7155], 0.227, [3.0374, -0.002...","([0.0, 0.0029, 0.9952], 0.224, [2.9577, -0.002...","([0.0, 0.0048, 0.8977], 0.224, [3.0158, -0.002...","([0.0, 0.0032, 0.7152, 0.9492], 0.227, [3.0359...","([0.0, 0.0071, 0.7326, 0.9516], 0.227, [3.0133...","([0.0, 0.0075, 0.9505, 0.8879], 0.224, [3.0242...","([0.0, 0.0106, 0.7384, 0.9654, 0.9689], 0.227,..."
3,헬스&뷰티,"([0.0, 0.0037], 0.207, [9.3215, -0.0086])","([0.0, 0.0004, 0.0081], 0.349, [11.7714, -0.01...","([0.0, 0.0053, 0.5535], 0.214, [9.1679, -0.008...","([0.0009, 0.0025, 0.3366], 0.227, [7.4088, -0....","([0.0, 0.0005, 0.0056, 0.2711], 0.371, [11.663...","([0.0001, 0.0, 0.0001, 0.0031], 0.494, [7.2965...","([0.0014, 0.0047, 0.821, 0.4358], 0.228, [7.54...","([0.0002, 0.0, 0.0002, 0.9587, 0.0068], 0.494,..."
4,임신/육아_비중,"([0.0, 0.0209], 0.136, [0.4863, -0.0005])","([0.0013, 0.0607, 0.0094], 0.286, [0.2921, -0....","([0.0, 0.0191, 0.5463], 0.145, [0.4989, -0.000...","([0.0222, 0.0199, 0.5918], 0.143, [0.3999, -0....","([0.001, 0.0485, 0.0064, 0.2708], 0.31, [0.300...","([0.0119, 0.1983, 0.0079, 0.3769], 0.302, [0.4...","([0.0481, 0.0141, 0.3746, 0.3981], 0.162, [0.3...","([0.0269, 0.1475, 0.0093, 0.4109, 0.6126], 0.3..."


In [82]:
# 대구 편의점 휴일 군집2
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekend_seg2_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekend_seg2_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekend_seg2_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekend_seg2_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))
  
# 최종 Data Frame
print('대구 편의점 휴일 군집2-주거상권')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구휴일군집2-주거상권.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 휴일 군집2-주거상권


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,식사,"([0.0, 0.0218], 0.134, [23.5689, -0.0068])","([0.0, 0.0012, 0.0008], 0.368, [26.6808, -0.00...","([0.0, 0.0246, 0.9652], 0.134, [23.5806, -0.00...","([0.0, 0.0695, 0.2036], 0.173, [26.1747, -0.00...","([0.0, 0.0017, 0.0009, 0.6013], 0.373, [26.630...","([0.0, 0.0025, 0.0021, 0.7015], 0.37, [26.0791...","([0.0, 0.1096, 0.6106, 0.1763], 0.179, [26.485...","([0.0, 0.0051, 0.0026, 0.6873, 0.8424], 0.374,..."
1,마실거리,"([0.0, 0.0002], 0.317, [26.9921, -0.015])","([0.0, 0.0005, 0.0014], 0.488, [23.2042, -0.01...","([0.0, 0.0002, 0.776], 0.318, [27.0888, -0.015...","([0.0, 0.0001, 0.2703], 0.34, [24.1284, -0.016...","([0.0, 0.0004, 0.0011, 0.3805], 0.499, [23.311...","([0.0, 0.0035, 0.0027, 0.6034], 0.492, [24.247...","([0.0, 0.0001, 0.426, 0.1876], 0.352, [23.5134...","([0.0, 0.003, 0.0032, 0.4682, 0.8384], 0.5, [2..."
2,임신/육아_비중,"([0.0, 0.0079], 0.176, [0.3045, -0.0002])","([0.0, 0.017, 0.287], 0.202, [0.2719, -0.0002,...","([0.0, 0.0083, 0.7484], 0.178, [0.3072, -0.000...","([0.0001, 0.0125, 0.9833], 0.176, [0.3032, -0....","([0.0, 0.0167, 0.2641, 0.6202], 0.207, [0.2736...","([0.0, 0.0607, 0.2293, 0.5576], 0.21, [0.3044,...","([0.0001, 0.0136, 0.723, 0.8717], 0.179, [0.29...","([0.0001, 0.0631, 0.2423, 0.7693, 0.6701], 0.2..."


In [83]:
# 대구 편의점 휴일 군집3
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekend_seg3_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekend_seg3_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekend_seg3_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekend_seg3_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 편의점 휴일 군집3-주거')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구휴일군집3-주거.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 휴일 군집3-주거


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh


In [84]:
# 대구 편의점 휴일 군집4
import statsmodels.api as sm

model_list = [] # 일반 단일 모델 (코로나 - 2020 카드)
model_list_t = [] # 코로나, 기온
model_list_r = [] # 코로나, 강수량
model_list_h = [] # 코로나, 습도
model_list_tr = [] # 코로나, 기온, 강수량
model_list_th = [] # 코로나, 기온, 습도
model_list_rh = [] # 코로나 강수량, 습도
model_list_trh = [] # 코로나 기온, 강수량, 습도

business_type = []
for j in gs_daegu_weekend_seg4_corr40_list:
  i = circulation_list_reverse[j]

  x2020 = i+'_y' # 종속변수 컬럼명

  # 잔차 항 - X 
  X1 = sm.add_constant(gs_daegu_weekend_seg4_corr['confirmed_y'])
  X2 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'temp_mean']])
  X3 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'rain_mean']])
  X4 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'hum_mean']])
  X5 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean']])
  X6 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'temp_mean', 'hum_mean']])
  X7 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'rain_mean', 'hum_mean']])
  X8 = sm.add_constant(gs_daegu_weekend_seg4_corr[['confirmed_y', 'temp_mean', 'rain_mean', 'hum_mean']])

  result1 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X1) 
  result2 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X2) 
  result3 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X3)
  result4 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X4) 
  result5 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X5) 
  result6 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X6)
  result7 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X7) 
  result8 = sm.OLS(gs_daegu_weekend_seg4_corr[x2020], X8)

  business_type.append(j)

  model_list.append((list(map(lambda x:np.round(x, float_list), result1.fit().pvalues)), np.round(result1.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result1.fit().params))))
  model_list_t.append((list(map(lambda x:np.round(x, float_list), result2.fit().pvalues)), np.round(result2.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result2.fit().params))))
  model_list_r.append((list(map(lambda x:np.round(x, float_list), result3.fit().pvalues)), np.round(result3.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result3.fit().params))))
  model_list_h.append((list(map(lambda x:np.round(x, float_list), result4.fit().pvalues)), np.round(result4.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result4.fit().params))))
  model_list_tr.append((list(map(lambda x:np.round(x, float_list), result5.fit().pvalues)), np.round(result5.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result5.fit().params))))
  model_list_th.append((list(map(lambda x:np.round(x, float_list), result6.fit().pvalues)), np.round(result6.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result6.fit().params))))
  model_list_rh.append((list(map(lambda x:np.round(x, float_list), result7.fit().pvalues)), np.round(result7.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result7.fit().params))))
  model_list_trh.append((list(map(lambda x:np.round(x, float_list), result8.fit().pvalues)), np.round(result8.fit().rsquared,3), list(map(lambda x:np.round(x, float_list), result8.fit().params))))

# 최종 Data Frame
print('대구 편의점 휴일 군집4-교통중심')
pvalue_test = pd.DataFrame(columns=['gs_type','c', 'ct', 'cr','ch', 'ctr','cth','crh','ctrh'], #
                           data = {'gs_type': business_type,
                                   'c': model_list, 'ct':model_list_t, 'cr':model_list_r, 'ch':model_list_h, 'ctr':model_list_tr,
                                   'cth':model_list_th, 'crh':model_list_rh, 'ctrh':model_list_trh
                                   })

gs_daegu_pvalue_test.append(pvalue_test)
pvalue_test.to_csv('/content/drive/My Drive/big_contest/model/편의점대구휴일군집4-교통중심.csv',index=False, encoding='cp949')
pvalue_test

대구 편의점 휴일 군집4-교통중심


,gs_type,c,ct,cr,ch,ctr,cth,crh,ctrh
0,마실거리,"([0.0, 0.0011], 0.255, [30.6759, -0.0131])","([0.0, 0.0023, 0.0001], 0.53, [25.9646, -0.01,...","([0.0, 0.0012, 0.8287], 0.256, [30.7511, -0.01...","([0.0, 0.0008, 0.3422], 0.273, [28.146, -0.014...","([0.0, 0.0019, 0.0, 0.317], 0.544, [26.0797, -...","([0.0, 0.024, 0.0, 0.2101], 0.551, [28.304, -0...","([0.0, 0.0008, 0.5149, 0.2625], 0.282, [27.628...","([0.0, 0.0217, 0.0001, 0.566, 0.3476], 0.556, ..."
